# Pseudo Shepardization with GPT-4
Starting with a root case, follow its subsequent citations, and for each one, fetch the court filing, extract the text surrounding the citation, and use gpt-4 to generate a structured data model representing the new case's impact to the root case citation.

In [1]:
import os
import json
import openai
from dotenv import load_dotenv
import pandas as pd
import requests
from datetime import datetime
from lxml import etree
import openai
import instructor
from tenacity import (
    stop_after_attempt, 
    wait_fixed,
)
from justopinion.download import CAPClient
from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

load_dotenv()
CAP_API_KEY = os.getenv('CAP_API_KEY')

In [2]:
os.chdir(os.path.dirname(os.getcwd()))

* Data comes from the Case Law Access project API, which provides 500 free calls per day

In [3]:
client = CAPClient(api_token=CAP_API_KEY)
response = client.fetch_id(cap_id=1079906, full_case=True)

In [4]:
case = json.loads(response.content)

In [5]:
print(case["casebody"]["data"]["head_matter"])

[No. D038121.
Fourth Dist., Div. One.
Apr. 30, 2002.]
RICHARD CUNNINGHAM et al., Plaintiffs and Respondents, v. UNIVERSAL UNDERWRITERS, Defendant and Appellant.
Counsel
Callahan, McCune & Willis and Colrena Kay Johnson for Defendant and Appellant.
Ault, Davis & Shonfeld, Thomas H. Ault, Gary E. Davis and R. Michael Jordan for Plaintiffs and Respondents.


In [6]:
opinion = case["casebody"]["data"]["opinions"][0]["text"]
opinion = opinion.replace("\n", "\n\n")
display(Markdown("**Case excerpt:**"))
display(Markdown(opinion[:3500]))

**Case excerpt:**

Opinion

HALLER, Acting P. J.

Cunningham Enterprises, Inc. (Cunningham), sued its liability insurer, Universal Underwriters (Universal), alleging Universal breached its duty to defend Cunningham in an action brought by a third party. After Cunningham and Universal each moved for summary judgment, the court denied Universal’s motion, granted Cunningham’s motion, and entered judgment in Cunningham’s favor for $197,837.59 plus costs. Universal appeals.

We conclude the undisputed facts establish the absence of a potential for coverage under the relevant insurance policies and therefore Universal had no duty to defend Cunningham in the underlying action. Thus, the trial court erred in granting Cunningham summary judgment and in denying Universal’s summary judgment motion. Accordingly, we reverse the judgment and remand for the court to enter summary judgment in Universal’s favor.

Factual Summary

This case concerns whether Universal had a duty to defend Cunningham in an action brought by Steven Beus and his limited liability company (collectively Beus). Beus’s complaint alleged the following:

In April 1996, Beus met with a real estate agent representing Cunningham regarding potential sites for an Isuzu dealership. The agent showed Beus property owned by Cunningham located on Johnson Avenue in El Cajon (the Johnson property). The Johnson property consisted of a front lot and a rear lot. “The rear lot was ‘improved’ with dilapidated and unsightly . . . buildings.” The front lot was improved with structures appropriate for use as an automobile dealership. Cunningham’s real estate agent told Beus that the tenant on the front lot, Holland Motor Homes, would vacate the premises in late November or early December.

In September 1996, Beus and Cunningham entered into a. written lease agreement pertaining to the Johnson property. Although the agreement specifically provided that the January 1, 1997 lease commencement date was “subject to . . . Holland . . . vacating the Premises,” Beus alleged that “the true intent of the parties was that the lease commencement date could be made earlier, but not later, than January 1, 1997.”

In reliance on representations that Holland would be vacating the property no later than January 1, 1997, Beus expended funds to open the dealership on that date, including entering into agreements with the local redevelopment agency, obtaining loans from Isuzu, and paying for the demolition of the buildings on the rear lot. Beus also ordered an inventory of new Isuzu vehicles to be delivered to the new dealership.

While waiting to occupy the main portion of the lot, Beus moved into the vacant back lot, and stored the new Isuzu vehicles at this lot. On December 28, 1996, Beus representatives attempted to take possession of the front portion of the lot. But Holland representatives said that Holland would not vacate the premises for another two weeks. In the succeeding six weeks, Beus made numerous demands for possession of the front portion of the Johnson property, receiving in return a series of promises and representations from Cunningham and others that Holland would vacate the premises by certain dates. Meanwhile, Beus attempted to “operate out of a damp, tin building remaining on the rear lot portion of the premises with no phones, no electricity, and no plumbing.” Because of its inability to move into the front lot, Beus was unable to obtain a sales license from the Department of Motor Vehicles, and thus was unable to

* Data model

In [7]:
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

class CitationAnalysis(BaseModel):
    """Information about a legal citation"""
    
    citation: str = Field(
        ...,
        description="The Citation specified by the user.",
    )
    legal_question: str = Field(
        ...,
        description="A concise and well-structured legal question. For example: 'Plaintiff slipped and fell in the hotel lobby. Is the hotel liable?'",
    )
    rule: str = Field(
        ...,
        description="A concise legal ruling, decision, or authority. For example: 'If a hotel knows its floors are wet, it has a duty to take reasonable steps to avoid such injury.'",
    )
    application: str = Field(
        ...,
        description="Application, or potential application of the rule. For example: 'The hotel acted negligently'."
    )
    citation_reference: str = Field(
        ...,
        description="A concise explanation of the way in which the citation was referenced or used in the context."
    )

In [8]:
client = instructor.patch(openai.OpenAI())

In [9]:
from tenacity import Retrying

def analyze_citation(cited_opinion_bluebook_citation: str, excerpt: str) -> CitationAnalysis:
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=CitationAnalysis,
        max_retries=Retrying(
        stop=stop_after_attempt(5),
        wait=wait_fixed(1),
    ),
        messages=[
            {
            "role": "system",
            "content": "Your role is to extract information about a legal citation using the context, which is an excerpt from a subsequent legal proceeding that referenced the citation of interest.",
            },
            {"role": "user", "content": f"Your task focuses on citation: **{cited_opinion_bluebook_citation}**"},
            {"role": "user", "content": f"Here is the context: {excerpt}"}
        ]
    )

* Main function

In [12]:
import requests
import json
from datetime import datetime
from lxml import etree
from typing import List, Dict, Any

def fetch_case(case_id: int) -> Dict[str, Any]:
    """
    Fetches a case by its ID from the Case Law API and returns the case data as a dictionary.

    Args:
        case_id (int): The unique identifier for the case.

    Returns:
        Dict[str, Any]: The case data as a dictionary.
    """
    BASE_URL = "https://api.case.law/v1"
    endpoint_path = f"/cases/{case_id}"
    response = requests.get(
        f"{BASE_URL}{endpoint_path}/?full_case=true&body_format=xml",
        headers={"Authorization": f"Token {CAP_API_KEY}"},
    )
    return json.loads(response.content)

def fetch_citing_case(forward_citation: Dict[str, Any]) -> Dict[str, Any]:
    """
    Fetches a citing case using the forward citation information.

    Args:
        forward_citation (Dict[str, Any]): The forward citation information containing the case ID.

    Returns:
        Dict[str, Any]: The citing case data as a dictionary.
    """
    return fetch_case(forward_citation["id"])

def fetch_forward_citations(case_id: int) -> List[Dict[str, Any]]:
    """
    Fetches forward citations for a given case ID.

    Args:
        case_id (int): The unique identifier for the case.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries containing forward citation data.
    """
    BASE_URL = "https://api.case.law/v1"
    endpoint_path = f"/cases/"
    response = requests.get(
        f"{BASE_URL}{endpoint_path}/?cites_to={case_id}",
        headers={"Authorization": f"Token {CAP_API_KEY}"},
    )
    return json.loads(response.content).get("results")

def get_bluebook_citation(case: Dict[str, Any]) -> str:
    """
    Generates a Bluebook citation for a given case.

    Args:
        case (Dict[str, Any]): The case data as a dictionary.

    Returns:
        str: The Bluebook citation as a string.
    """
    case_name = case["name_abbreviation"]
    reporter_info = case["citations"][0]["cite"]
    court = case["court"]["name_abbreviation"]
    decision_date = datetime.strptime(case["decision_date"], "%Y-%m-%d")
    return f"{case_name}, {reporter_info} ({court} {decision_date.year})"

def get_case_body(case: Dict[str, Any]) -> str:
    """
    Extracts the case body from the case data.

    Args:
        case (Dict[str, Any]): The case data as a dictionary.

    Returns:
        str: The case body as a string.
    """
    return case["casebody"]["data"]

def parse_case_body(case_body_xml: str) -> etree._Element:
    """
    Parses the XML case body and returns the root element.

    Args:
        case_body_xml (str): The case body as an XML string.

    Returns:
        etree._Element: The root element of the parsed XML.
    """
    return etree.fromstring(
        bytes(case_body_xml, "utf-8"), parser=etree.XMLParser(recover=True)
    )

def dump_elems(case_body_root_elem: etree._Element) -> List[etree._Element]:
    """
    Dumps all elements from the case body's root element.

    Args:
        case_body_root_elem (etree._Element): The root element of the case body.

    Returns:
        List[etree._Element]: A list of all elements in the case body.
    """
    return [elem for elem in case_body_root_elem.iter()]

def locate_citations(citing_case_elems: List[etree._Element], cited_case_id: int) -> List[int]:
    """
    Locates citations within a citing case and returns their locations.

    Args:
        citing_case_elems (List[etree._Element]): A list of elements from the citing case.
        cited_case_id (int): The ID of the cited case.

    Returns:
        List[int]: A list of locations where the cited case is referenced.
    """
    locations = []
    for location_counter, elem in enumerate(citing_case_elems):
        if elem.tag.endswith("extracted-citation"):
            case_id_info = elem.attrib.get("case-ids")
            if case_id_info is not None:
                citing_case_ids = case_id_info.split(",")
                for case_id_str in citing_case_ids:
                    if int(case_id_str) == cited_case_id:
                        locations.append(location_counter)
    return locations

def get_excerpt(
    citing_case_elems: List[etree._Element],
    location: int,
    length_before: int = 12,
    length_after: int = 8,
) -> str:
    """
    Extracts an excerpt from the citing case elements based on a specified location.

    Args:
        citing_case_elems (List[etree._Element]): A list of elements from the citing case.
        location (int): The location index of the citation.
        length_before (int, optional): The number of elements to include before the citation. Defaults to 15.
        length_after (int, optional): The number of elements to include after the citation. Defaults to 15.

    Returns:
        str: The extracted excerpt as a string.
    """
    excerpt_elems = citing_case_elems[
        max(0, location - length_before) : min(
            location + length_after, len(citing_case_elems)
        )
    ]
    excerpt_text = []
    for elem in excerpt_elems:
        elem_text = [text for text in elem.itertext()]
        excerpt_text.extend(elem_text)
    return " ".join(excerpt_text)

def get_text(
    citing_case_elems: List[etree._Element],
) -> str:
    """
    Extracts the full text from the citing case elements.

    Args:
        citing_case_elems (List[etree._Element]): A list of elements from the citing case.

    Returns:
        str: The full text extracted from the citing case elements.
    """
    excerpt_text = []
    for elem in citing_case_elems:
        elem_text = [text for text in elem.itertext()]
        excerpt_text.extend(elem_text)
    return " ".join(excerpt_text)

def pseudo_shepardize(case_id: int) -> List[Dict[str, Any]]:
    """
    Performs a pseudo-Shepardization process on a case by analyzing its forward citations.

    Args:
        case_id (int): The unique identifier for the case.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries containing analysis results for each citing case.
    """
    result = []
    case = fetch_case(case_id)
    bluebook_citation = get_bluebook_citation(case)
    forward_citations = fetch_forward_citations(case_id)
    for citation in forward_citations:
        citing_case = fetch_citing_case(citation)
        citing_case_id = citing_case["id"]
        citing_case_bluebook_citation = get_bluebook_citation(citing_case)
        print(f"Analyzing citing case: {citing_case_bluebook_citation} ...")
        citing_case_body_xml = get_case_body(citing_case)
        citing_case_root_elem = parse_case_body(citing_case_body_xml)
        citing_case_elems = dump_elems(citing_case_root_elem)
        citing_locations = locate_citations(citing_case_elems, case_id)
        for location in citing_locations:
            excerpt = get_excerpt(citing_case_elems, location)
            text = get_text(citing_case_elems)
            citation_model_result: CitationAnalysis = analyze_citation(bluebook_citation, excerpt)
            one_result = {
                "case_id": case_id,
                "bluebook_citation": bluebook_citation,
                "citing_case_id": citing_case_id,
                "citing_case_bluebook_citation": citing_case_bluebook_citation,
                "excerpt": excerpt,
                "citation": citation_model_result.citation,
                "legal_question": citation_model_result.legal_question,
                "rule": citation_model_result.rule,
                "application": citation_model_result.application,
                "citation_reference": citation_model_result.citation_reference,
                "text": text,
            }
            result.append(one_result)
    return result

In [13]:
result = pseudo_shepardize(case_id=1079906)

Analyzing citing case: Brucia v. Hartford Accident & Indemnity, 307 F. Supp. 2d 1079 (N.D. Cal. 2003) ...
Analyzing citing case: Baroco West, Inc. v. Scottsdale Insurance, 110 Cal. App. 4th 96 (Cal. Ct. App. 2003) ...
Analyzing citing case: Mercury Insurance v. Ayala, 116 Cal. App. 4th 1198 (Cal. Ct. App. 2004) ...
Analyzing citing case: F & H Construction v. ITT Hartford Insurance, 118 Cal. App. 4th 364 (Cal. Ct. App. 2004) ...
Analyzing citing case: Davis v. Farmers Insurance Group, 134 Cal. App. 4th 100 (Cal. Ct. App. 2005) ...
Analyzing citing case: Kim Seng Co. v. Great American Insurance, 179 Cal. App. 4th 1030 (Cal. Ct. App. 2009) ...
Analyzing citing case: Demarco v. Everest Indemnity Insurance, 370 F. App'x 795 (9th Cir. 2010) ...
Analyzing citing case: Lakeland Village Homeowners Ass'n v. Great American Insurance Group, 727 F. Supp. 2d 887 (E.D. Cal. 2010) ...
Analyzing citing case: BTR Hampstead, LLC v. Source Interlink Distribution, LLC, 194 Md. App. 538 (Md. Ct. Spec. App.

In [3]:
df = pd.read_parquet("pseudo_shepardize.parquet")

In [4]:
# df = pd.DataFrame(result)
df.head()

,case_id,bluebook_citation,citing_case_id,citing_case_bluebook_citation,excerpt,citation,legal_question,rule,application,citation_reference,text
0,1079906,"Cunningham v. Universal Underwriters, 98 Cal. ...",9271539,"Brucia v. Hartford Accident & Indemnity, 307 F...","For the foregoing reasons, the Court GRANTS Pl...","Cunningham v. Universal Underwriters, 98 Cal. ...",Does damage to a leasehold interest constitute...,A tenant’s right to possess property on the le...,In the context of insurance policies specifyin...,The citation was used to support the argument ...,"\n Joseph BRUCIA and Frances Brucia, Plainti..."
1,1079906,"Cunningham v. Universal Underwriters, 98 Cal. ...",648755,"Baroco West, Inc. v. Scottsdale Insurance, 110...",17 Cal.4th 38 70 Cal.Rptr.2d 118 948 P.2d 909 ...,"Cunningham v. Universal Underwriters, 98 Cal. ...",What is the legal question addressed or implie...,,,The context provided repeatedly references Cun...,"\n [No. E032011. \n Fourth Dist., Div. Two..."
2,1079906,"Cunningham v. Universal Underwriters, 98 Cal. ...",462638,"Mercury Insurance v. Ayala, 116 Cal. App. 4th ...",2. Coverage: Which limit applies? \n \n...,"Cunningham v. Universal Underwriters, 98 Cal. ...","In the context of uninsured motorist coverage,...",If the terms of an insurance policy are unambi...,The language of the insurance policy in questi...,The citation was referenced to support the pri...,"\n [No. B165390. \n Second Dist., Div. Fiv..."
3,1079906,"Cunningham v. Universal Underwriters, 98 Cal. ...",1377175,"F & H Construction v. ITT Hartford Insurance, ...","*377 Collin v. American Empire Ins. Co., supra...","Cunningham v. Universal Underwriters, 98 Cal. ...",Does welding lower grade pile caps to driven p...,Welding lower grade pile caps to driven piles ...,The court considered whether F&H suffered prop...,The court referenced Cunningham v. Universal U...,\n [No. C042707. \n Third Dist. \n May 5...
4,1079906,"Cunningham v. Universal Underwriters, 98 Cal. ...",6048633,"Davis v. Farmers Insurance Group, 134 Cal. App...",11 Cal.4th 1 44 Cal.Rptr.2d 370 900 P.2d 619 (...,"Cunningham v. Universal Underwriters, 98 Cal. ...",When does an insurer have a duty to defend act...,An insurer must defend any action that seeks d...,This citation establishes that an insurer is o...,This citation was used to illustrate when an i...,"\n [No. D044724. \n Fourth Dist., Div. One..."


In [15]:
df.to_parquet("pseudo_shepardize.parquet", index=False)
df.to_excel("pseudo_shepardize.xlsx", index=False)

In [5]:
display_df = df[df["citing_case_bluebook_citation"]!="Baroco West, Inc. v. Scottsdale Insurance, 110 Cal. App. 4th 96 (Cal. Ct. App. 2003)"]
display_df = display_df[display_df["citing_case_bluebook_citation"]!="Kim Seng Co. v. Great American Insurance, 179 Cal. App. 4th 1030 (Cal. Ct. App. 2009)"]

In [6]:
Markdown(display_df[[
    "citing_case_bluebook_citation", 
    "legal_question", 
    "rule", 
    "application", 
    "citation_reference",
]].to_markdown(index=False))

| citing_case_bluebook_citation                                                                             | legal_question                                                                                                                                                                                                                 | rule                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | application                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | citation_reference                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|:----------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Brucia v. Hartford Accident & Indemnity, 307 F. Supp. 2d 1079 (N.D. Cal. 2003)                            | Does damage to a leasehold interest constitute "property damage" as defined in insurance policy terms, specifically in cases where the tenant has not yet taken possession of the property?                                    | A tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession merely gives the tenant a right to abandon the tenancy and sue for damages, but does not grant the tenant the legal right to act as a possessor of the real property until actual transfer. Therefore, damage to a leasehold interest, in cases where possession has not occurred, does not constitute damage to a tangible property right. | In the context of insurance policies specifying coverage for "property damage" limited to "tangible property," damage to a leasehold interest, where the tenant has not yet taken possession, falls outside the scope of property damage. This is due to the characterization of a tenant’s interest prior to possession as a contractual rather than a tangible property right.                                                                                                                                                                                                        | The citation was used to support the argument that damage to a leasehold interest, prior to the tenant taking possession, does not constitute 'property damage' under the defined terms of an insurance policy. It clarifies the legal status of a leasehold interest as initially a contractual right, which does not transform into a tangible property right until actual possession occurs.                                                                                                                                                                                                                                                                |
| Mercury Insurance v. Ayala, 116 Cal. App. 4th 1198 (Cal. Ct. App. 2004)                                   | In the context of uninsured motorist coverage, does the per person limit or the per accident limit apply to the appellant's claims? In this case, does the language of the insurance policy clearly define which limit applies | If the terms of an insurance policy are unambiguous, the plain meaning of those terms should be given effect.                                                                                                                                                                                                                                                                                                                                                                                                                                                  | The language of the insurance policy in question was deemed unambiguous by the court, and it was determined that the per person limit applies to the appellant's claims in line with the insurance company's interpretation.                                                                                                                                                                                                                                                                                                                                                            | The citation was referenced to support the principle that if insurance policy language is unambiguous, the plain meaning of those terms should be given effect. It was used to justify the application of the per person limit in the case at hand, agreeing with the insurance company's interpretation.                                                                                                                                                                                                                                                                                                                                                      |
| F & H Construction v. ITT Hartford Insurance, 118 Cal. App. 4th 364 (Cal. Ct. App. 2004)                  | Does welding lower grade pile caps to driven piles constitute physical injury to tangible property when the only injury is the structure’s failure to perform as intended?                                                     | Welding lower grade pile caps to driven piles does not constitute physical injury to tangible property where the only injury shown is the welded structure’s failure to perform as intended.                                                                                                                                                                                                                                                                                                                                                                   | The court considered whether F&H suffered property damage under the definition of loss of use of tangible property that is not physically injured. It concluded that costs for repairing and modifying defective caps and for loss of early completion bonus are unrelated to the rental value and do not qualify as property damage under the commercial liability insurance policy.                                                                                                                                                                                                   | The court referenced Cunningham v. Universal Underwriters to support the conclusion that cost of repairs and modifications for defective workmanship does not qualify as property damage covered by commercial liability insurance policy, affirming that contractors and developers cannot obtain liability insurance for inferior or defective workmanship - a risk that is not covered by commercial liability insurance. It was used to strengthen the court's reasoning for granting summary judgment in favor of Hartford, indicating that F&H failed to establish it suffered property damage within the meaning of the CGLI policy issued by Hartford. |
| Davis v. Farmers Insurance Group, 134 Cal. App. 4th 100 (Cal. Ct. App. 2005)                              | When does an insurer have a duty to defend actions against its policy holder?                                                                                                                                                  | An insurer must defend any action that seeks damages potentially within the coverage of the policy; however, no duty to defend exists when the third party complaint can by no conceivable theory raise a single issue that could bring it within the policy coverage.                                                                                                                                                                                                                                                                                         | This citation establishes that an insurer is obligated to defend any lawsuit that seeks damages which could potentially fall within the coverage scope of the policy. However, if the lawsuit could not conceivable raise any issue that would fall under the policy's coverage, then the insurer has no defense obligation.                                                                                                                                                                                                                                                            | This citation was used to illustrate when an insurer has a duty to defend a lawsuit against its policy holder. Particularly, it highlights the principle that an insurer must defend actions that potentially seek damages within the policy's coverage, while also noting that no duty exists when the complaint cannot raise any issues that would fit under this coverage.                                                                                                                                                                                                                                                                                  |
| Demarco v. Everest Indemnity Insurance, 370 F. App'x 795 (9th Cir. 2010)                                  | Does the insurance policy require the insurer to defend against a suit seeking damages for an act, error, or omission that occurs in the conduct of the Named Insured’s operations?                                            | The insurance policy provides that the insurer has a duty to defend against any suit seeking damages for an act, error or omission that "occurs in the conduct of the Named Insured’s operations."                                                                                                                                                                                                                                                                                                                                                             | The citation does not directly address an instance of misrepresentation within Great Alarm's operations, such as negotiating stock purchases, but supports the broader context of assessing insurance coverage obligations based on the scope of the insured's operations.                                                                                                                                                                                                                                                                                                              | The citation is referenced to support determining the scope of an insurance company's duty to defend based on the operations of the insured as defined within the policy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| Lakeland Village Homeowners Ass'n v. Great American Insurance Group, 727 F. Supp. 2d 887 (E.D. Cal. 2010) | In a duty to defend case, what are the required evidentiary standards for the insured and the insurer when moving for summary judgment?                                                                                        | An insured moving for summary judgment in a duty to defend case must only show the existence of a potential for coverage, meaning that the underlying claim may fall within policy coverage. Conversely, an insurer moving for summary judgment must establish the absence of any potential for coverage, indicating that the underlying complaint can by no conceivable theory raise a single issue which could bring it within the policy coverage.                                                                                                          | This sets disparate evidentiary standards for the insured and the insurer in summary judgment motions concerning duty to defend cases, favoring the insured's ability to demonstrate potential coverage under the policy.                                                                                                                                                                                                                                                                                                                                                               | Cunningham v. Universal Underwriters is cited in the context as establishing the differing burdens of proof required for insureds and insurers when moving for summary judgment in duty to defend cases. This delineation clarifies that an insured must merely show a potential for coverage, whereas an insurer must conclusively prove the absence of any potential for coverage.                                                                                                                                                                                                                                                                           |
| BTR Hampstead, LLC v. Source Interlink Distribution, LLC, 194 Md. App. 538 (Md. Ct. Spec. App. 2010)      | What constitutes an actual eviction and what acts by the landlord could be considered as such?                                                                                                                                 | An eviction is defined as the act or process of legally dispossessing a person of land or rental property. A wrongful eviction occurs when the person recovering the property had no right to dispossess the other party from the property.                                                                                                                                                                                                                                                                                                                    | In the context of landlord and tenant relations, wrongful eviction occurs when the landlord, without the legal right to do so, dispossesses the tenant of the rental property. This can include acts that physically remove the tenant or prevent the tenant's access to the property, such as changing locks, refusing entry, padlocking entryways, or threats of violence.                                                                                                                                                                                                            | The case is referenced to affirm the definition of an eviction as the process of legally dispossessing a person of land or rental property, with a focus on wrongful eviction as occurring when there is no legal right to dispossess the other party from the property. The citation plays a role in understanding the legal concept of eviction within the broader discussion on landlord and tenant disputes over actual eviction.                                                                                                                                                                                                                          |
| Ginsberg v. Gamson, 205 Cal. App. 4th 873 (Cal. Ct. App. 2012)                                            | What is the legal distinction between a claim for breach of the covenant of quiet enjoyment and a constructive eviction claim?                                                                                                 | A claim for breach of the covenant of quiet enjoyment can be maintained while the tenant remains in possession of the property, whereas a constructive eviction claim must not be brought until the tenant has vacated the property.                                                                                                                                                                                                                                                                                                                           | In Cunningham v. Universal Underwriters, the court held that while both claims deal with interference by the landlord, the inherent difference lies in the status of tenancy - whether the tenant still occupies the property or has left due to eviction.                                                                                                                                                                                                                                                                                                                              | The citation was used to clarify the essential distinction between a breach of the covenant of quiet enjoyment and a constructive eviction claim, highlighting that for the latter, the tenant must have vacated the property before bringing the claim.                                                                                                                                                                                                                                                                                                                                                                                                       |
| Imperium Insurance v. Unigard Insurance, 16 F. Supp. 3d 1104 (E.D. Cal. 2014)                             | What must an insurer demonstrate to attain summary judgment in its favor on the basis of either lack of coverage under the insurance policy or because the contested claim falls within an exclusion under that policy?        | An insurer seeking summary judgment must establish the absence of any potential for coverage, meaning that the underlying complaint cannot, under any conceivable theory, raise a single issue which might bring it within the policy coverage; or, when the claim is said to be excluded, the insurer must prove that the claim unequivocally falls within an exclusion.                                                                                                                                                                                      | In the context of an insurer seeking summary judgment on either the basis of lack of coverage or because the claim falls within an exclusion, the insurer carries the burden of proof. Specifically, the insurer must demonstrate conclusively that there is no possibility of coverage under the policy or that the claim definitely falls within a policy exclusion. Any ambiguity or doubt concerning the duty to defend should be resolved in favor of the insured, indicating a preference towards granting coverage or defense under the policy unless it is clearly unwarranted. | The case was cited to support the principle that, when an insurer moves for summary judgment based on the claim of no coverage or an exclusion, the insurer has the burden to unequivocally prove that the claim cannot possibly fit within the scope of the policy coverage or definitely falls within an exclusion. This standard was referenced from the case Montrose Chemical Corporation v. Superior Court, enforcing the stringent criteria an insurer must meet to attain a summary judgment in their favor.                                                                                                                                           |
| Thee Sombrero, Inc. v. Scottsdale Ins. Co., 239 Cal. Rptr. 3d 416 (Cal. App. Ct. 2018)                    | Does a tenant have a tangible property interest in leased premises before taking possession?                                                                                                                                   | Before a tenant takes possession, the tenant's right to possess the premises is not considered tangible property; it is viewed as a contractual right that does not mature into a property right until possession actually occurs.                                                                                                                                                                                                                                                                                                                             | In the context of insurance and discussions of tangible property interests, the tenant's pre-possession rights are considered contractual and intangible, contrasting with the tangible nature of property interests that comes with possession.                                                                                                                                                                                                                                                                                                                                        | The citation was used in making a distinction between the tangible property interest associated with actual possession of leased premises and the intangible nature of a pre-possession contractual right to occupy such premises.                                                                                                                                                                                                                                                                                                                                                                                                                             |
| Mcmillin Homes Constr., Inc. v. Nat'l Fire & Marine Ins. Co., 247 Cal. Rptr. 3d 825 (Cal. App. Ct. 2019)  | When a policy term has been judicially construed, is it still considered ambiguous?                                                                                                                                            | Where a policy term has been judicially construed, it becomes unambiguous.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | The court applied this rule to the context of the CCC exclusion, indicating it was not ambiguous given its judicial construction.                                                                                                                                                                                                                                                                                                                                                                                                                                                       | The case was cited in support of the principle that a term that has been judicially construed is not considered ambiguous, further noting that the judicial construction of the term should be incorporated into the policy unless the parties express a contrary intent.                                                                                                                                                                                                                                                                                                                                                                                      |
| Gov't Emps. Ins. Co. v. Nadkarni, 391 F. Supp. 3d 917 (N.D. Cal. 2019)                                    | In the context of insurance coverage, does the actual departure of a tenant pursuant to an eviction notice constitute wrongful eviction?                                                                                       | Physical dispossession (actual departure of a tenant pursuant to an eviction notice) is the sine qua non of eviction.                                                                                                                                                                                                                                                                                                                                                                                                                                          | In insurance-coverage cases, the focus is on the actual departure of the tenant, following an eviction notice, as the critical element determining what constitutes eviction.                                                                                                                                                                                                                                                                                                                                                                                                           | Clarified that physical dispossession is the sine qua non (essential condition) of eviction in the context of insurance coverage disputes regarding wrongful eviction.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| Gov't Emps. Ins. Co. v. Nadkarni, 391 F. Supp. 3d 917 (N.D. Cal. 2019)                                    | Do the defendants' references to 'personal injury' and other enumerated torts extend the eviction into the policy period?                                                                                                      | Coverage is based on when the 'offense is committed.'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | In the context of insurance coverage, for torts like wrongful eviction, the offense is considered committed when the tenants departed or when the defendants regained possession of the property.                                                                                                                                                                                                                                                                                                                                                                                       | Cunningham v. Universal Underwriters was used to support the principle that insurance coverage for enumerated torts such as wrongful eviction is determined by when the offense was committed, which is either when the tenants departed or when the property owners regained possession.                                                                                                                                                                                                                                                                                                                                                                      |

In [7]:
display_df["Excerpt"] = display_df["excerpt"].replace("\n", "\n\n")

In [8]:
from src.citations import create_annotated_text
from src.parsing.utils import clean_whitespace

In [9]:
display_df["Excerpt"] = display_df["Excerpt"].apply(lambda x: clean_whitespace(x))
display_df["Excerpt"] = display_df["Excerpt"].apply(lambda x: create_annotated_text(x))
display_df["citing_case_bluebook_citation"] = display_df["citing_case_bluebook_citation"].apply(lambda x: create_annotated_text(x))

In [10]:
formatted_strings = [
    f"### {row['citing_case_bluebook_citation']}\n\n* {row['legal_question']}\n\n* {row['rule']}\n\n* {row['application']}\n\n* {row['citation_reference']}\n\n#### Excerpt:\n\n{row['Excerpt']}\n\n___\n\n"
    for index, row in display_df.iterrows()
]
formatted_string = "\n\n".join(formatted_strings)

Markdown(formatted_string)

### Brucia v. Hartford Accident & Indemnity, <a href="https://cite.case.law/f-supp-2d/307/1079"><a class="citation" href="https://cite.case.law/f-supp-2d/307/1079" title="307 F. Supp. 2d 1079">307 F. Supp. 2d 1079</a></a> (N.D. Cal. 2003)

* Does damage to a leasehold interest constitute "property damage" as defined in insurance policy terms, specifically in cases where the tenant has not yet taken possession of the property?

* A tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession merely gives the tenant a right to abandon the tenancy and sue for damages, but does not grant the tenant the legal right to act as a possessor of the real property until actual transfer. Therefore, damage to a leasehold interest, in cases where possession has not occurred, does not constitute damage to a tangible property right.

* In the context of insurance policies specifying coverage for "property damage" limited to "tangible property," damage to a leasehold interest, where the tenant has not yet taken possession, falls outside the scope of property damage. This is due to the characterization of a tenant’s interest prior to possession as a contractual rather than a tangible property right.

* The citation was used to support the argument that damage to a leasehold interest, prior to the tenant taking possession, does not constitute 'property damage' under the defined terms of an insurance policy. It clarifies the legal status of a leasehold interest as initially a contractual right, which does not transform into a tangible property right until actual possession occurs.

#### Excerpt:

For the foregoing reasons, the Court GRANTS Plaintiffs’ motion for partial summary judgment and DENIES The Hartford’s Motion to Dismiss. IT IS SO ORDERED. . Plaintiffs have also argued that the allegations in the Underlying Complaint cannot be claims for "property damage,” since the property damage section of the policy applies only to "tangible property.” Plaintiffs argue that a leasehold interest is not "tangible property” and therefore damage to a leasehold interest is not property damage as defined in the policy. No California court has directly considered whether a tenant in possession of a property has a tangible property right. In Kazi v. State Farm Fire and Casualty Co., <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/cal-4th/24/871" title="24 Cal. 4th 871">24 Cal.4th 871</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> (2001), the Supreme Court of California held that an easement represents only a nonposses-sory right to use another’s property, and therefore is not tangible property. <a class="citation" href="https://cite.case.law/p3d/15/223#p881" title="15 P.3d 223, 881">Id. at 881</a>, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/p3d/15/223"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> . In Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002), a California appellate court applied the reasoning of Kazi to hold that a nonpossessing tenant, whose claimed loss concerned solely the interference with its right to possession of real property under the parties’ lease, did not allege damage to a tangible property right. As the court in Cunningham explained, "[a] tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer.” <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162, 1155">Id. at 1155</a>-1156, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> . The clear implication of the Cunningham court’s reasoning is that when a tenant legally takes possession of the premises, the tenant, unlike an easement holder or a nonpossessing tenant, has a possessory property right in the premises. As such, a tenant in possession presumably has a tangible property interest in the premises and can assert claims for property damage, as defined in the insurance contract. Regardless, this argument is not dispositive in the instant case, since the Court finds that R & H has asserted claims for personal injury that extend beyond property damage. . Plaintiffs have also argued that the allegations in the Underlying Complaint cannot be claims for "property damage,” since the property damage section of the policy applies only to "tangible property.” Plaintiffs argue that a leasehold interest is not "tangible property” and therefore damage to a leasehold interest is not property damage as defined in the policy. No California court has directly considered whether a tenant in possession of a property has a tangible property right. In Kazi v. State Farm Fire and Casualty Co., <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/cal-4th/24/871" title="24 Cal. 4th 871">24 Cal.4th 871</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> (2001), the Supreme Court of California held that an easement represents only a nonposses-sory right to use another’s property, and therefore is not tangible property. <a class="citation" href="https://cite.case.law/p3d/15/223#p881" title="15 P.3d 223, 881">Id. at 881</a>, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/p3d/15/223"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> . In Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002), a California appellate court applied the reasoning of Kazi to hold that a nonpossessing tenant, whose claimed loss concerned solely the interference with its right to possession of real property under the parties’ lease, did not allege damage to a tangible property right. As the court in Cunningham explained, "[a] tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer.” <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162, 1155">Id. at 1155</a>-1156, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> . The clear implication of the Cunningham court’s reasoning is that when a tenant legally takes possession of the premises, the tenant, unlike an easement holder or a nonpossessing tenant, has a possessory property right in the premises. As such, a tenant in possession presumably has a tangible property interest in the premises and can assert claims for property damage, as defined in the insurance contract. Regardless, this argument is not dispositive in the instant case, since the Court finds that R & H has asserted claims for personal injury that extend beyond property damage. Kazi v. State Farm Fire and Casualty Co., <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/cal-4th/24/871" title="24 Cal. 4th 871">24 Cal.4th 871</a></a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> <a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">Id.</a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Kazi Cunningham <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">Id.</a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Cunningham

___



### Mercury Insurance v. Ayala, <a href="https://cite.case.law/cal-app-4th/116/1198"><a class="citation" href="https://cite.case.law/cal-app-4th/116/1198" title="116 Cal. App. 4th 1198">116 Cal. App. 4th 1198</a></a> (Cal. Ct. App. 2004)

* In the context of uninsured motorist coverage, does the per person limit or the per accident limit apply to the appellant's claims? In this case, does the language of the insurance policy clearly define which limit applies

* If the terms of an insurance policy are unambiguous, the plain meaning of those terms should be given effect.

* The language of the insurance policy in question was deemed unambiguous by the court, and it was determined that the per person limit applies to the appellant's claims in line with the insurance company's interpretation.

* The citation was referenced to support the principle that if insurance policy language is unambiguous, the plain meaning of those terms should be given effect. It was used to justify the application of the per person limit in the case at hand, agreeing with the insurance company's interpretation.

#### Excerpt:

2. Coverage: Which limit applies?

2

Coverage: Which limit applies? 2 2 ' Part IV of appellants’ policy is entitled “Uninsured Motorists Coverage.” In it, Mercury promises to pay “all sums which the insured . . . shall be legally entitled to recover as damages from the owner, or operator of an uninsured motor vehicle because of bodily injury, sustained by the insured, caused by accident and arising out of the . . . use of such uninsured motor vehicles . . . .” ' The next part of the policy, titled “Part IV—Conditions,” includes the provision that “The limit of liability stated in the declarations as applicable to ‘each person’ is the limit of the company’s liability for all damages arising out of bodily injury sustained by one person in any one accident.... For the purposes of this provision, the ‘bodily injury sustained by any one person’ as used herein, shall be deemed to include all injury and damages for care, loss of consortium and injury to any interpersonal relationship sustained by others as a consequence of such bodily injury.” The second quoted sentence is in boldface. At summary judgment, Mercury contended that the unambiguous language of this clause meant that the per person limit of $15,000, not the per accident limit of $30,000, applies to appellants’ claims. Appellants make two claims, that the per accident limit applies under the terms of the policy and that the per accident limit must apply, because Insurance Code section 11580.2 mandates that it does. As to the policy itself, we agree with Mercury that the language is unambiguous. We thus give effect to its plain meaning (Cunningham v. *1202 Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141#p1149"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1149 [ <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ]), which is clearly the one Mercury ascribes to it—the per person limit applies. Numerous cases beginning with United Services Automobile Assn. v. Warner (1976) <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://cite.case.law/cal-app-3d/64/957" title="64 Cal. App. 3d 957">64 Cal.App.3d 957</a></a> [ <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./135/34" title="135 Cal. Rptr. 34">135 Cal.Rptr. 34</a></a> ] have construed similar language in liability policies and have found that the language is unambiguous and means what Mercury, and we, say that it means. (Cunningham *1202 Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141120"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> United Services Automobile Assn. v. Warner <a href="https://cite.case.law/cal-app-3d/64/957135"><a class="citation" href="https://cite.case.law/cal-app-3d/64/957" title="64 Cal. App. 3d 957">64 Cal.App.3d 957</a></a> <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./135/34" title="135 Cal. Rptr. 34">135 Cal.Rptr. 34</a></a> Appellants seek a different result, citing Abellon v. Hartford Ins. Co. (1985) <a href="https://cite.case.law/cal-app-3d/167/21"><a class="citation" href="https://cite.case.law/cal-app-3d/167/21" title="167 Cal. App. 3d 21">167 Cal.App.3d 21</a></a> [ <a href="https://cite.case.law/cal-app-3d/167/21"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./212/852" title="212 Cal. Rptr. 852">212 Cal.Rptr. 852</a></a> ]. Abellon, like Warner, is a declaratory relief action filed by an insurer after its insured was in a car accident and was sued not only by the other driver, but by that driver’s spouse on a loss of consortium claim. Neither case involves uninsured motorist coverage, but in both cases, the insurer sought a declaration that the per person, not the per accident, limit of the liability coverage applied. Warner found that the per person limit applied, and Abellon found that the per accident limit applied. However, in Warner, the policy language was like the language before us, 3 and in Abellon there was no similar language defining “bodily injury to any one person” to include the loss of consortium sustained by another. Instead, the insured in Abellon case had “no notice when it purchased the policy that loss of consortium damages fell within the purview of the ‘per person’ limitation.” (Abellon, at p. 31.) Abellon does not help us construe appellants’ policy. Abellon v. Hartford Ins. Co. <a href="https://cite.case.law/cal-app-3d/167/21"><a class="citation" href="https://cite.case.law/cal-app-3d/167/21" title="167 Cal. App. 3d 21">167 Cal.App.3d 21</a></a>

___



### F & H Construction v. ITT Hartford Insurance, <a href="https://cite.case.law/cal-app-4th/118/364"><a class="citation" href="https://cite.case.law/cal-app-4th/118/364" title="118 Cal. App. 4th 364">118 Cal. App. 4th 364</a></a> (Cal. Ct. App. 2004)

* Does welding lower grade pile caps to driven piles constitute physical injury to tangible property when the only injury is the structure’s failure to perform as intended?

* Welding lower grade pile caps to driven piles does not constitute physical injury to tangible property where the only injury shown is the welded structure’s failure to perform as intended.

* The court considered whether F&H suffered property damage under the definition of loss of use of tangible property that is not physically injured. It concluded that costs for repairing and modifying defective caps and for loss of early completion bonus are unrelated to the rental value and do not qualify as property damage under the commercial liability insurance policy.

* The court referenced Cunningham v. Universal Underwriters to support the conclusion that cost of repairs and modifications for defective workmanship does not qualify as property damage covered by commercial liability insurance policy, affirming that contractors and developers cannot obtain liability insurance for inferior or defective workmanship - a risk that is not covered by commercial liability insurance. It was used to strengthen the court's reasoning for granting summary judgment in favor of Hartford, indicating that F&H failed to establish it suffered property damage within the meaning of the CGLI policy issued by Hartford.

#### Excerpt:

*377 Collin v. American Empire Ins. Co., supra, Traveler’s We therefore hold that welding the lower grade pile caps to the driven piles does not constitute physical injury to tangible property where the only injury shown is the welded structure’s failure to perform as intended. E. Loss of Use of Tangible Property Loss of Use of Tangible Property We next consider whether F&H suffered “property damage” under the second definition of that term, i.e. “[ljoss of use of tangible property that is not physically injured.” The court in Collin v. American Empire Ins. Co., supra, 21 Cal.App.4th at page 818, explained the meaning of the phrase “loss of use,” quoting from 23 California Jurisprudence Third: “ ‘The measure of damages for the loss of use of personal property may be determined with reference to the rental value of similar property which the plaintiff can hire for use during the period when he is deprived of the use of his own property.’ (23 Cal.Jur.3d, Damages, § 69, pp. 129-130 . . . .)” (Italics omitted.) Collin v. American Empire Ins. Co., supra, F&H does not seek damages for the rental value (or its equivalent) for the loss of the use of the Los Vaqueros facility during the time period modifications were made to the pile caps, or even for a period of time caused by delay. The likely reason for this is the fact the project was completed on time. As noted, the only costs claimed by F&H are the costs for repairing and modifying the defective caps and for loss of the early completion bonus. Those costs are unrelated to rental value. F&H has therefore failed to establish its damages are covered by the policy as “property damage.” A contrary conclusion would allow contractors and developers to obtain liability insurance for inferior or defective workmanship, a risk not covered by commercial liability insurance. (Maryland Casualty Co. v. Reeder, supra, 221 Cal.App.3d at pp. 967-968; see also Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1156 [ <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ].) Accordingly, we conclude the trial court properly granted summary judgment in favor of Hartford on the grounds F&H failed to establish it suffered property damage within the meaning of the CGLI policy issued by Hartford. (Maryland Casualty Co. v. Reeder, supra, Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> *378 DISPOSITION *378 The summary judgment in favor of ITT Hartford Insurance Company of the Midwest is affirmed. As the prevailing party, Hartford is awarded its costs on appeal. (Cal. Rules of Court, rule 27(a)(1).) Morrison, J., and Robie, J., concurred. Hartford Insurance Company of the Midwest was erroneously sued as ITT Hartford Insurance Group, Inc. Hartford Insurance Company of the Midwest was erroneously sued as ITT Hartford Insurance Group, Inc.

___



### Davis v. Farmers Insurance Group, <a href="https://cite.case.law/cal-app-4th/134/100"><a class="citation" href="https://cite.case.law/cal-app-4th/134/100" title="134 Cal. App. 4th 100">134 Cal. App. 4th 100</a></a> (Cal. Ct. App. 2005)

* When does an insurer have a duty to defend actions against its policy holder?

* An insurer must defend any action that seeks damages potentially within the coverage of the policy; however, no duty to defend exists when the third party complaint can by no conceivable theory raise a single issue that could bring it within the policy coverage.

* This citation establishes that an insurer is obligated to defend any lawsuit that seeks damages which could potentially fall within the coverage scope of the policy. However, if the lawsuit could not conceivable raise any issue that would fall under the policy's coverage, then the insurer has no defense obligation.

* This citation was used to illustrate when an insurer has a duty to defend a lawsuit against its policy holder. Particularly, it highlights the principle that an insurer must defend actions that potentially seek damages within the policy's coverage, while also noting that no duty exists when the complaint cannot raise any issues that would fit under this coverage.

#### Excerpt:

<a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://cite.case.law/cal-4th/11/1" title="11 Cal. 4th 1">11 Cal.4th 1</a></a> <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> <a href="https://cite.case.law/p2d/900/619"><a class="citation" href="https://cite.case.law/p2d/900/619" title="900 P.2d 619">900 P.2d 619</a></a> (Waller).) (E.M.M.I., Inc. Zurich American Ins. Co. <a href="https://cite.case.law/cal-4th/32/465"><a class="citation" href="https://cite.case.law/cal-4th/32/465" title="32 Cal. 4th 465">32 Cal.4th 465</a></a> <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/9/701"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/9/701" title="9 Cal. Rptr. 3d 701">9 Cal.Rptr.3d 701</a></a> <a href="https://cite.case.law/p3d/84/385"><a class="citation" href="https://cite.case.law/p3d/84/385" title="84 P.3d 385">84 P.3d 385</a></a> (Waller, supra, (<a class="citation" href="https://cite.case.law/p3d/84/385" title="84 P.3d 385">Id.</a> (Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> An insurer must defend any action that seeks damages potentially within the coverage of the policy; however, no duty to defend exists when the third party complaint “ ‘can by no conceivable theory raise a single issue [that] could bring it within the policy coverage.’ ” (Montrose Chemical Corp. v. Superior Court (1993) <a href="https://cite.case.law/cal-4th/6/287#p299"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> , 299-300 [ <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> ], quoting Gray v. Zurich Insurance Co. (1966) <a href="https://cite.case.law/cal-2d/65/263"><a class="citation" href="https://cite.case.law/cal-2d/65/263" title="65 Cal. 2d 263">65 Cal.2d 263</a></a> , 276, fn. 15 [ <a href="https://cite.case.law/cal-2d/65/263"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./54/104" title="54 Cal. Rptr. 104">54 Cal.Rptr. 104</a></a> , <a href="https://cite.case.law/cal-2d/65/263"><a class="citation" href="https://cite.case.law/p2d/419/168" title="419 P.2d 168">419 P.2d 168</a></a> ] (Gray), italics omitted.) Whether a third party action asserts a potentially covered claim under the policy triggering the duty to defend requires us to interpret the language of the insuring agreement and presents a question of law. (Alex Robertson Co. v. Imperial Casualty & Indemnity Co. (1992) <a href="https://cite.case.law/cal-app-4th/8/338#p342"><a class="citation" href="https://cite.case.law/cal-app-4th/8/338" title="8 Cal. App. 4th 338">8 Cal.App.4th 338</a></a> , 342-343 [ <a href="https://cite.case.law/cal-app-4th/8/338"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/10/165" title="10 Cal. Rptr. 2d 165">10 Cal.Rptr.2d 165</a></a> ]; Union Oil Co. v. International Ins. Co. (1995) <a href="https://cite.case.law/cal-app-4th/37/930"><a class="citation" href="https://cite.case.law/cal-app-4th/37/930" title="37 Cal. App. 4th 930">37 Cal.App.4th 930</a></a> , 936 [ <a href="https://cite.case.law/cal-app-4th/37/930"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/4" title="44 Cal. Rptr. 2d 4">44 Cal.Rptr.2d 4</a></a> ].) An insurer’s duty to defend turns on the facts alleged in the underlying complaint or available to the insurer from extrinsic sources at the time the claim is made. (Montrose Chemical Corp. v. Superior Court, supra, <a class="citation" href="https://cite.case.law/cal-4th/6/287#p295" title="6 Cal. 4th 287, 295">6 Cal.4th at p. 295</a>; Gray, supra, <a class="citation" href="https://cite.case.law/cal-2d/65/263#p276" title="65 Cal. 2d 263, 276">65 Cal.2d at p. 276</a>.) The insured has the burden of showing that the claim falls within the scope of coverage and the insurer has the burden of proving that an otherwise covered claim is barred by a policy exclusion. (Travelers Casualty & Surety Co. v. Superior Court (1998) <a href="https://cite.case.law/cal-app-4th/63/1440"><a class="citation" href="https://cite.case.law/cal-app-4th/63/1440" title="63 Cal. App. 4th 1440">63 Cal.App.4th 1440</a></a> , 1453 [ <a href="https://cite.case.law/cal-app-4th/63/1440"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/75/54" title="75 Cal. Rptr. 2d 54">75 Cal.Rptr.2d 54</a></a> ].) “[I]n an action [in which] none of the claims is even potentially covered, the insurer does not have a duty to defend.” (Buss v. Superior Court (1997) <a href="https://cite.case.law/cal-4th/16/35"><a class="citation" href="https://cite.case.law/cal-4th/16/35" title="16 Cal. 4th 35">16 Cal.4th 35</a></a> , 47 [ <a href="https://cite.case.law/cal-4th/16/35"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/65/366" title="65 Cal. Rptr. 2d 366">65 Cal.Rptr.2d 366</a></a> , <a href="https://cite.case.law/cal-4th/16/35"><a class="citation" href="https://cite.case.law/p2d/939/766" title="939 P.2d 766">939 P.2d 766</a></a> ].) (Montrose Chemical Corp. Superior Court <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/p2d/861/1153"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a>

___



### Demarco v. Everest Indemnity Insurance, <a href="https://cite.case.law/f-appx/370/795"><a class="citation" href="https://cite.case.law/f-appx/370/795" title="370 F. App'x 795">370 F. App'x 795</a></a> (9th Cir. 2010)

* Does the insurance policy require the insurer to defend against a suit seeking damages for an act, error, or omission that occurs in the conduct of the Named Insured’s operations?

* The insurance policy provides that the insurer has a duty to defend against any suit seeking damages for an act, error or omission that "occurs in the conduct of the Named Insured’s operations."

* The citation does not directly address an instance of misrepresentation within Great Alarm's operations, such as negotiating stock purchases, but supports the broader context of assessing insurance coverage obligations based on the scope of the insured's operations.

* The citation is referenced to support determining the scope of an insurance company's duty to defend based on the operations of the insured as defined within the policy.

#### Excerpt:

<a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> <a href="https://cite.case.law/p2d/846/792"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> <a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">Id.</a> <a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">Id.</a> Montrose Chon. Corp. v. Super. Ct., <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> Horace Mann, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> 846 P.2d at 795 Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> see also Waller v. Truck Ins. Exck, Inc., <a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://cite.case.law/cal-4th/11/1" title="11 Cal. 4th 1">11 Cal.4th 1</a></a> <a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> <a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://cite.case.law/p2d/900/619" title="900 P.2d 619">900 P.2d 619</a></a> Here, the relevant policy provides that Everest has a duty to defend against any suit seeking damages for an act, error or omission that “occurs in the conduct of the Named Insured’s operations.” Greater Alarm’s “operations” are defined in the policy as residential and commercial burglar and fire alarm installation and monitoring, medical emergency systems installation and monitoring, “C.C.T.V. installation/service/repair,” and access control. Although some of the alleged misrepresentations were about Great Alarm’s operations, negotiating the stock purchase was not “in the conduct of’ those operations. See Waller, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> , <a class="citation" href="https://cite.case.law/p2d/900/619#p627" title="900 P.2d 619, 627">900 P.2d at 627</a> . Accordingly, we conclude that the arbitration demand does not “reveal a possibility” of coverage under the policy, and thus, Everest had no duty to defend.

___



### Lakeland Village Homeowners Ass'n v. Great American Insurance Group, <a href="https://cite.case.law/f-supp-2d/727/887"><a class="citation" href="https://cite.case.law/f-supp-2d/727/887" title="727 F. Supp. 2d 887">727 F. Supp. 2d 887</a></a> (E.D. Cal. 2010)

* In a duty to defend case, what are the required evidentiary standards for the insured and the insurer when moving for summary judgment?

* An insured moving for summary judgment in a duty to defend case must only show the existence of a potential for coverage, meaning that the underlying claim may fall within policy coverage. Conversely, an insurer moving for summary judgment must establish the absence of any potential for coverage, indicating that the underlying complaint can by no conceivable theory raise a single issue which could bring it within the policy coverage.

* This sets disparate evidentiary standards for the insured and the insurer in summary judgment motions concerning duty to defend cases, favoring the insured's ability to demonstrate potential coverage under the policy.

* Cunningham v. Universal Underwriters is cited in the context as establishing the differing burdens of proof required for insureds and insurers when moving for summary judgment in duty to defend cases. This delineation clarifies that an insured must merely show a potential for coverage, whereas an insurer must conclusively prove the absence of any potential for coverage.

#### Excerpt:

Scottsdale Ins. Co. v. MV Transp., <a href="https://cite.case.law/cal-4th/36/643#p654"><a class="citation" href="https://cite.case.law/cal-4th/36/643" title="36 Cal. 4th 643">36 Cal.4th 643</a></a> , 654, <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/31/147" title="31 Cal. Rptr. 3d 147">31 Cal.Rptr.3d 147</a></a> , <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://cite.case.law/p3d/115/460" title="115 P.3d 460">115 P.3d 460</a></a> (2005) (citation omitted). Further, “[a]ny doubt as to whether the facts give rise to a duty to defend is resolved in the insured’s favor.” Horace Mann Ins. Co. v. Barbara B., <a href="https://cite.case.law/cal-4th/4/1076#p1081"><a class="citation" href="https://cite.case.law/cal-4th/4/1076" title="4 Cal. 4th 1076">4 Cal.4th 1076</a></a> , 1081, <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> , <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> (1993). Scottsdale Ins. Co. v. MV Transp., <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://cite.case.law/cal-4th/36/643" title="36 Cal. 4th 643">36 Cal.4th 643</a></a> <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/31/147" title="31 Cal. Rptr. 3d 147">31 Cal.Rptr.3d 147</a></a> <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://cite.case.law/p3d/115/460" title="115 P.3d 460">115 P.3d 460</a></a> Horace Mann Ins. Co. v. Barbara B., 4 <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/cal-4th/4/1076" title="4 Cal. 4th 1076">4 Cal.4th 1076</a></a> <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> In a duty to defend case, an insured moving for summary judgment “need only show ‘the existence of a potential for coverage,’ i.e., ‘that the underlying claim may fall within policy coverage.’ ” Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141#p1147"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1147, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002) (citing Montrose Chem. Corp. v. Sup. Ct., <a href="https://cite.case.law/cal-4th/6/287#p300"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> , 300, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> (1993)). Whereas, an insurer moving for summary judgment “ ‘must establish the absence of any ... potential’ for coverage, i.e., that the underlying complaint ‘can by no conceivable theory raise a single issue which could bring *891 it within the policy coverage.’ ” <a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">Id.</a> (citing Montrose Chem. Corp., at 300, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> (1993)). “Although the insurer’s burden in moving for summary judgment is greater than the insured’s burden in bringing its own affirmative motion, this disparity merely reflects the substantive law.” <a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">Id.</a> (citation omitted). Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Montrose Chem. Corp. v. Sup. Ct., <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> *891 Id.

___



### BTR Hampstead, LLC v. Source Interlink Distribution, LLC, <a href="https://cite.case.law/md-app/194/538"><a class="citation" href="https://cite.case.law/md-app/194/538" title="194 Md. App. 538">194 Md. App. 538</a></a> (Md. Ct. Spec. App. 2010)

* What constitutes an actual eviction and what acts by the landlord could be considered as such?

* An eviction is defined as the act or process of legally dispossessing a person of land or rental property. A wrongful eviction occurs when the person recovering the property had no right to dispossess the other party from the property.

* In the context of landlord and tenant relations, wrongful eviction occurs when the landlord, without the legal right to do so, dispossesses the tenant of the rental property. This can include acts that physically remove the tenant or prevent the tenant's access to the property, such as changing locks, refusing entry, padlocking entryways, or threats of violence.

* The case is referenced to affirm the definition of an eviction as the process of legally dispossessing a person of land or rental property, with a focus on wrongful eviction as occurring when there is no legal right to dispossess the other party from the property. The citation plays a role in understanding the legal concept of eviction within the broader discussion on landlord and tenant disputes over actual eviction.

#### Excerpt:

sub judice. 2 2 any *555 Our review of Maryland case law has disclosed no definition of “actual eviction.” Black’s Law Dictionary 35 (6th ed.1990) defines actual eviction as “[a]n actual expulsion of the tenant out of all or some part of the demised premises. A physical ouster or dispossession from the very thing granted or some substantial part thereof.” (Emphasis added). Thompson on Real Property § 41.03(c)(2), entitled “Actual Eviction by the Landlord,” explains: dispossession from the very thing granted or some substantial part thereof.” The landlord will be in breach of the implied covenant of quiet enjoyment if the landlord, or someone for whose acts the landlord is responsible, wrongfully ousts the tenant from the leased premises. The cases refer to this conduct, which can take the form of physical expulsion or a physical exclusion of the tenant from the leased premises, as an actual eviction. Particular acts by the landlord that have been found to be sufficient to constitute actual eviction of a tenant have included changing the locks on the leased premises, refusing to permit the tenant to enter a building in which space has been leased, padlocking the entry to the leased premises, and taking possession of the leased premises with threats of violence against the tenant or members of the tenant’s family if they attempt to reenter. The cases also recognize that the landlord’s eviction efforts may deprive the tenant of only a portion of the leased premises, leaving the tenant in possession of the remainder. If the landlord deprives the tenant of a significant portion of the leased premises, the landlord’s conduct will generally be regarded as constituting a partial actual eviction, and a breach of the implied covenant. (Emphasis added) (footnotes omitted); see also Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal. Rptr.2d 162</a></a> , 168-69 (2002) (“The legal definition of an eviction is the same: an ‘eviction’ is ‘[t]he act or process of legally dispossessing a person of land or rental property.’ (Black’s *556 Law Diet. (7th ed.1999) p. 575, col. 2.) A wrongful eviction thus occurs when the person recovering the property had no right to dispossess the other party from the property.” (Alteration in original)); cf. Day v. Watson, <a href="https://cite.case.law/mich/8/535#p536"><a class="citation" href="https://cite.case.law/mich/8/535" title="8 Mich. 535">8 Mich. 535</a></a> , 536 (1860) (“[T]he entry being followed by a continuous possession, which was inconsistent with the possessory title assured to the tenants under the lease, that possession amounts very clearly to an eviction.”). also Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal. Rptr.2d 162</a></a> *556 wrongful cf. Day v. Watson, <a href="https://cite.case.law/mich/8/535"><a class="citation" href="https://cite.case.law/mich/8/535" title="8 Mich. 535">8 Mich. 535</a></a> The Restatement (Second) of Property: Landlord and Tenant § 6.1 (hereinafter “the Restatement”), provides in pertinent part: Landlord and Tenant

___



### Ginsberg v. Gamson, <a href="https://cite.case.law/cal-app-4th/205/873"><a class="citation" href="https://cite.case.law/cal-app-4th/205/873" title="205 Cal. App. 4th 873">205 Cal. App. 4th 873</a></a> (Cal. Ct. App. 2012)

* What is the legal distinction between a claim for breach of the covenant of quiet enjoyment and a constructive eviction claim?

* A claim for breach of the covenant of quiet enjoyment can be maintained while the tenant remains in possession of the property, whereas a constructive eviction claim must not be brought until the tenant has vacated the property.

* In Cunningham v. Universal Underwriters, the court held that while both claims deal with interference by the landlord, the inherent difference lies in the status of tenancy - whether the tenant still occupies the property or has left due to eviction.

* The citation was used to clarify the essential distinction between a breach of the covenant of quiet enjoyment and a constructive eviction claim, highlighting that for the latter, the tenant must have vacated the property before bringing the claim.

#### Excerpt:

<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./99/86" title="99 Cal. Rptr. 86">99 Cal.Rptr. 86</a> However, in Guntert v. City of Stockton (1976) <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://cite.case.law/cal-app-3d/55/131" title="55 Cal. App. 3d 131">55 Cal.App.3d 131</a></a> [ <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">126 Cal.Rptr. 690</a></a> ] (Guntert), the court distinguished claims in which the landlord has actually or constructively ousted the tenant from those in which the landlord’s interference with the tenant’s enjoyment or use of the property does not lead to ouster. The Guntert court recognized that a tenant has alternative remedies to respond to the landlord’s interference and, instead of vacating, may “stand upon the lease and sue for damages.” (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690, 140">Id. at p. 140</a>.) In Guntert, the plaintiff tenant had not quit the premises, despite the landlord’s service of an invalid and bad faith eviction notice. The court thus concluded: “[The tenant] is not suing for a wrongful, constructive eviction but for the lessor’s unjustified and unauthorized interference with his profitable use of the leased property. The rule requiring ouster or surrender prior to suit for wrongful eviction does not preclude the tenant from his election to stand upon the lease, remain in possession and sue for breach of contract damages.” (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690, 141">Id. at p. 141</a>, italics added.) Guntert v. City of Stockton <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://cite.case.law/cal-app-3d/55/131" title="55 Cal. App. 3d 131">55 Cal.App.3d 131</a></a> <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">126 Cal.Rptr. 690</a></a> (Guntert), Guntert (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">Id.</a> Guntert, for breach of contract damages.” (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">Id.</a> Subsequent cases have similarly drawn a distinction between a wrongful eviction claim, and one in which the tenant remains in possession of the premises and sues for damages arising out of the landlord’s interference with the tenant’s beneficial use of the leased premises. For example, in Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1153 [ <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ], the court explained: “While a claim for breach of the covenant of quiet enjoyment is similar to a constructive eviction claim, the critical difference is that the latter claim may not be brought until the tenant has vacated the property.” Thus, breach of the implied covenant of quiet enjoyment can be understood as a title encompassing claims for wrongful eviction, and also claims in which the tenant’s use of the premises is disturbed, but the tenant remains in possession. Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> b. Courts have only attached tort liability to wrongful eviction claims Courts have only attached tort liability to wrongful eviction claims It is true, as Ginsberg asserts, that some courts have implicitly or explicitly indicated a tenant may recover punitive damages in connection with a claim for breach of the implied covenant of quiet enjoyment, or identified the claim as a tort. However, all of those cases describe wrongful eviction claims. In other words, they are cases in which the implied covenant of quiet enjoyment *899 is breached by a wrongful eviction. For example, Ginsberg relies on Barkett v. Brucato (1953) <a href="https://cite.case.law/cal-app-2d/122/264"><a class="citation" href="https://cite.case.law/cal-app-2d/122/264" title="122 Cal. App. 2d 264">122 Cal.App.2d 264</a></a> [ <a href="https://cite.case.law/cal-app-2d/122/264"><a class="citation" href="https://cite.case.law/p2d/264/978" title="264 P.2d 978">264 P.2d 978</a></a> ] (Barkett). In Barkett, the landlord engaged in a series of egregious acts intended to force the residential tenants to vacate the premises, such as authorizing the removal of the building’s roof during the rainy season, leading to extensive flooding in the tenants’ apartment. (Id. at pp. 271-272.) The tenants eventually vacated the flat. (Id. at p. 272.) They then filed suit, alleging claims for breach of the covenant of quiet enjoyment, negligence, and “conspiracy to willfully and maliciously harass” the tenants. (Id. at p. 266.) A jury returned a general verdict against the landlord, and awarded $5,000 in damages, but did not award any punitive damages. (Ibid.) *899 Barkett Brucato

___



### Imperium Insurance v. Unigard Insurance, <a href="https://cite.case.law/f-supp-3d/16/1104"><a class="citation" href="https://cite.case.law/f-supp-3d/16/1104" title="16 F. Supp. 3d 1104">16 F. Supp. 3d 1104</a></a> (E.D. Cal. 2014)

* What must an insurer demonstrate to attain summary judgment in its favor on the basis of either lack of coverage under the insurance policy or because the contested claim falls within an exclusion under that policy?

* An insurer seeking summary judgment must establish the absence of any potential for coverage, meaning that the underlying complaint cannot, under any conceivable theory, raise a single issue which might bring it within the policy coverage; or, when the claim is said to be excluded, the insurer must prove that the claim unequivocally falls within an exclusion.

* In the context of an insurer seeking summary judgment on either the basis of lack of coverage or because the claim falls within an exclusion, the insurer carries the burden of proof. Specifically, the insurer must demonstrate conclusively that there is no possibility of coverage under the policy or that the claim definitely falls within a policy exclusion. Any ambiguity or doubt concerning the duty to defend should be resolved in favor of the insured, indicating a preference towards granting coverage or defense under the policy unless it is clearly unwarranted.

* The case was cited to support the principle that, when an insurer moves for summary judgment based on the claim of no coverage or an exclusion, the insurer has the burden to unequivocally prove that the claim cannot possibly fit within the scope of the policy coverage or definitely falls within an exclusion. This standard was referenced from the case Montrose Chemical Corporation v. Superior Court, enforcing the stringent criteria an insurer must meet to attain a summary judgment in their favor.

#### Excerpt:

65 Cal.2d at 267 54 Cal-Rptr. 104 <a href="https://cite.case.law/p2d/419/168"><a class="citation" href="https://cite.case.law/p2d/419/168" title="419 P.2d 168">419 P.2d 168</a></a> Montrose, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/p2d/861/1153"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> Waller, 11 Cal.4th at 19 <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> <a href="https://cite.case.law/p2d/900/619"><a class="citation" href="https://cite.case.law/p2d/900/619" title="900 P.2d 619">900 P.2d 619</a></a> An insurer moving for summary judgment “ ‘must establish the absence of any ... potential’ for coverage, i.e., that the underlying complaint ‘can by no conceivable theory raise a single issue which could bring it within the policy coverage.’ ” Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141#p1147"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1147, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002) (quoting Montrose Chem. Corp. v. Superior Court, <a href="https://cite.case.law/cal-4th/6/287#p300"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> , 300, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> (1993)). When an insurer seeks summary judgment “on the ground the claim is excluded,” the insurer has the burden “to prove that the claim falls within an exclusion.” Roberts v. Assurance Co. of America, <a href="https://cite.case.law/cal-app-4th/163/1398#p1406"><a class="citation" href="https://cite.case.law/cal-app-4th/163/1398" title="163 Cal. App. 4th 1398">163 Cal.App.4th 1398</a></a> , 1406, <a href="https://cite.case.law/cal-app-4th/163/1398"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/78/361" title="78 Cal. Rptr. 3d 361">78 Cal.Rptr.3d 361</a></a> (2008) (quotations and citations omitted). “Any doubt as to whether the facts give rise to a duty to defend is resolved in the insured’s favor.” Horace Mann Ins. Co. v. Barbara B., <a href="https://cite.case.law/cal-4th/4/1076#p1081"><a class="citation" href="https://cite.case.law/cal-4th/4/1076" title="4 Cal. 4th 1076">4 Cal.4th 1076</a></a> , 1081, <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> , <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> (1993). Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Montrose Chem. Corp. v. Superior Court, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> Roberts v. Assurance Co. of America, <a href="https://cite.case.law/cal-app-4th/163/1398"><a class="citation" href="https://cite.case.law/cal-app-4th/163/1398" title="163 Cal. App. 4th 1398">163 Cal.App.4th 1398</a></a>

___



### Thee Sombrero, Inc. v. Scottsdale Ins. Co., <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/239/416"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/239/416" title="239 Cal. Rptr. 3d 416">239 Cal. Rptr. 3d 416</a></a> (Cal. App. Ct. 2018)

* Does a tenant have a tangible property interest in leased premises before taking possession?

* Before a tenant takes possession, the tenant's right to possess the premises is not considered tangible property; it is viewed as a contractual right that does not mature into a property right until possession actually occurs.

* In the context of insurance and discussions of tangible property interests, the tenant's pre-possession rights are considered contractual and intangible, contrasting with the tangible nature of property interests that comes with possession.

* The citation was used in making a distinction between the tangible property interest associated with actual possession of leased premises and the intangible nature of a pre-possession contractual right to occupy such premises.

#### Excerpt:

Hendrickson a loss of the use of their land Hendrickson v. Zurich American Ins. Co. of Illinois supra <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/85/622"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/85/622" title="85 Cal. Rptr. 2d 622">85 Cal.Rptr.2d 622</a></a> Third, we question IPA's statement that "a right to occupy premises is not a tangible property interest." ( Scottsdale Ins. Co. v. International Protective Agency, Inc. , supra , 105 Wash.App. at p. 250 [19 P.3d at p. 1061].) At least under California law, "[a] lease is ... a conveyance of an estate in real property .... [Citation.]" ( Avalon Pacific-Santa Ana, L.P. v. HD Supply Repair & Remodel, LLC (2011) <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://cite.case.law/cal-app-4th/192/1183" title="192 Cal. App. 4th 1183">192 Cal.App.4th 1183</a></a> , 1190, <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/122/417" title="122 Cal. Rptr. 3d 417">122 Cal.Rptr.3d 417</a></a> .) A building is tangible. Dirt is tangible. Hence, a lessee in possession has a tangible property interest in the leased premises. (Compare Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141#p1155"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1155-1156, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> [before tenant takes possession, tenant's right to possess premises is not tangible property; it "is a contractual right that does not mature into a property right until possession actually occurs."] with Riverbank Holding Co., LLC v. N.H. Ins. Co. (E.D. Cal. 2012) 2012 WL 2119046 , at *8, 2012 U.S.Dist. LEXIS 78781 , at *22 [distinguishing Cunningham ; tenant has tangible property interest in leased premises after tenant "actually took possession of the property."].) Scottsdale Ins. Co. v. International Protective Agency, Inc. supra Avalon Pacific-Santa Ana, L.P. v. HD Supply Repair & Remodel, LLC <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://cite.case.law/cal-app-4th/192/1183" title="192 Cal. App. 4th 1183">192 Cal.App.4th 1183</a></a> <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/122/417" title="122 Cal. Rptr. 3d 417">122 Cal.Rptr.3d 417</a></a> Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Riverbank Holding Co., LLC v. N.H. Ins. Co. 2012 WL 2119046 2012 U.S.Dist. LEXIS 78781 Cunningham In any event, the issue is not whether, as a technical legal matter, a leasehold is tangible property. Rather, it is whether an insured, reading his or her policy, would understand "tangible property" to include real property that he or she leases. If your leased apartment was rendered uninhabitable by some noxious stench, you would conclude that you had lost the use of tangible property; and if a lawyer said no, actually you had merely lost the use of your intangible lease, you would goggle in disbelief. Most important, though, our view on this point is dictum, because our case is distinguishable. Here, Sombrero is the owner of the property, not a lessee. As such, it plainly has an interest in tangible property.

___



### Mcmillin Homes Constr., Inc. v. Nat'l Fire & Marine Ins. Co., <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/247/825"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/247/825" title="247 Cal. Rptr. 3d 825">247 Cal. Rptr. 3d 825</a></a> (Cal. App. Ct. 2019)

* When a policy term has been judicially construed, is it still considered ambiguous?

* Where a policy term has been judicially construed, it becomes unambiguous.

* The court applied this rule to the context of the CCC exclusion, indicating it was not ambiguous given its judicial construction.

* The case was cited in support of the principle that a term that has been judicially construed is not considered ambiguous, further noting that the judicial construction of the term should be incorporated into the policy unless the parties express a contrary intent.

#### Excerpt:

Having been judicially construed, the CCC exclusion is not ambiguous Where a policy term has been judicially construed, it is not ambiguous. ( County of San Diego v. Ace Property & Casualty Ins. Co. (2005) <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://cite.case.law/cal-4th/37/406" title="37 Cal. 4th 406">37 Cal.4th 406</a></a> , 423, <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/33/583" title="33 Cal. Rptr. 3d 583">33 Cal.Rptr.3d 583</a></a> , <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://cite.case.law/p3d/118/607" title="118 P.3d 607">118 P.3d 607</a></a> ( County of San Diego ).) "[T]he judicial construction of the term should be read into the policy unless the parties express a contrary intent." ( Bartlome v. State Farm (1989) <a href="https://cite.case.law/cal-app-3d/208/1235"><a class="citation" href="https://cite.case.law/cal-app-3d/208/1235" title="208 Cal. App. 3d 1235">208 Cal.App.3d 1235</a></a> , 1239, <a href="https://cite.case.law/cal-app-3d/208/1235"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./256/719" title="256 Cal. Rptr. 719">256 Cal.Rptr. 719</a></a> ( Bartlome ), accord, Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1150, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ; County of San Diego v. Ace Property & Casualty Ins. Co. <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/40633/583"><a class="citation" href="https://cite.case.law/cal-4th/37/406" title="37 Cal. 4th 406">37 Cal.4th 406</a></a> <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/33/583" title="33 Cal. Rptr. 3d 583">33 Cal.Rptr.3d 583</a></a> <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://cite.case.law/p3d/118/607" title="118 P.3d 607">118 P.3d 607</a></a> County of San Diego Bartlome v. State Farm <a href="https://cite.case.law/cal-app-3d/208/1235256"><a class="citation" href="https://cite.case.law/cal-app-3d/208/1235" title="208 Cal. App. 3d 1235">208 Cal.App.3d 1235</a></a> <a href="https://cite.case.law/cal-app-3d/208/1235"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./256/719" title="256 Cal. Rptr. 719">256 Cal.Rptr. 719</a></a> Bartlome Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141120"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> *1053 Norris v. Pacific Indem. Co. (1952) <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/cal-2d/39/420" title="39 Cal. 2d 420">39 Cal.2d 420</a></a> , 424, <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/p2d/247/1" title="247 P.2d 1">247 P.2d 1</a></a> ( Norris ) [provisions "should be given a meaning settled by judicial decision"].) This rule is applied " 'with caution, first determining whether the context in which the construed term is analogous ....' " ( Qualcomm, Inc. v. Certain Underwriters at Lloyd's, London (2008) <a href="https://cite.case.law/cal-app-4th/161/184"><a class="citation" href="https://cite.case.law/cal-app-4th/161/184" title="161 Cal. App. 4th 184">161 Cal.App.4th 184</a></a> , 201, <a href="https://cite.case.law/cal-app-4th/161/184"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/73/770" title="73 Cal. Rptr. 3d 770">73 Cal.Rptr.3d 770</a></a> .) As we explain, the CCC exclusion has been judicially construed in a sufficiently analogous context to require exclusive or complete control. ( Davis, supra , 79 Cal.App.3d at p. 872, <a href="https://www.courtlistener.com/c/Cal.%20Rptr./145/158"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./145/158" title="145 Cal. Rptr. 158">145 Cal.Rptr. 158</a></a> .) The additional insured's mere status as general contractor-with overall responsibility for and nominal control of the entire project-does not meet this standard. *1053 Norris v. Pacific Indem. Co. <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/cal-2d/39/420" title="39 Cal. 2d 420">39 Cal.2d 420</a></a> <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/p2d/247/1" title="247 P.2d 1">247 P.2d 1</a></a> Norris

___



### Gov't Emps. Ins. Co. v. Nadkarni, <a href="https://cite.case.law/f-supp-3d/391/917"><a class="citation" href="https://cite.case.law/f-supp-3d/391/917" title="391 F. Supp. 3d 917">391 F. Supp. 3d 917</a></a> (N.D. Cal. 2019)

* In the context of insurance coverage, does the actual departure of a tenant pursuant to an eviction notice constitute wrongful eviction?

* Physical dispossession (actual departure of a tenant pursuant to an eviction notice) is the sine qua non of eviction.

* In insurance-coverage cases, the focus is on the actual departure of the tenant, following an eviction notice, as the critical element determining what constitutes eviction.

* Clarified that physical dispossession is the sine qua non (essential condition) of eviction in the context of insurance coverage disputes regarding wrongful eviction.

#### Excerpt:

Harbor Ins. Co. v. Cent. Nat'l Ins. Co. <a href="https://cite.case.law/cal-app-3d/165/1029"><a class="citation" href="https://cite.case.law/cal-app-3d/165/1029" title="165 Cal. App. 3d 1029">165 Cal. App. 3d 1029</a></a> <a href="https://cite.case.law/cal-app-3d/165/1029"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./211/902" title="211 Cal. Rptr. 902">211 Cal.Rptr. 902</a></a> Here, as GEICO argues, at least three dates are relevant: March 13, 2018 (when the defendants served the notice of termination of tenancy); April 21, 2018 (when the plaintiffs vacated the property pursuant to the notice); and May 1, 2018 (when the defendants recovered possession of the property). 35 All dates have legal significance, and all occurred before the policy's effective date. For example, cases hold that wrongful eviction occurs when the landlord serves the eviction notice. Sylve v. Riley , <a href="https://cite.case.law/cal-app-4th/15/23#p26"><a class="citation" href="https://cite.case.law/cal-app-4th/15/23" title="15 Cal. App. 4th 23">15 Cal. App. 4th 23</a></a> , 26, <a href="https://cite.case.law/cal-app-4th/15/23"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/18/608" title="18 Cal. Rptr. 2d 608">18 Cal.Rptr.2d 608</a></a> (1993) ; Menefee v. Ostawari, <a href="https://cite.case.law/cal-app-3d/228/239#p246"><a class="citation" href="https://cite.case.law/cal-app-3d/228/239" title="228 Cal. App. 3d 239">228 Cal. App. 3d 239</a></a> , 246, <a href="https://cite.case.law/cal-app-3d/228/239"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./278/805" title="278 Cal. Rptr. 805">278 Cal.Rptr. 805</a></a> (1991) ("Appellant was damaged at the time of service of the 30-day notice in that respondents interfered with his continued right of possession at that time"). In insurance-coverage cases, courts emphasize the actual departure of the tenant pursuant to an eviction notice. Cunningham v. Universal Underwriters , <a href="https://cite.case.law/cal-app-4th/98/1141#p1149"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal. App. 4th 1141</a></a> , 1149, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002) (physical dispossession is the sine qua non of eviction); see Swain v. California Casualty Insurance Company, <a href="https://cite.case.law/cal-app-4th/99/1#p10"><a class="citation" href="https://cite.case.law/cal-app-4th/99/1" title="99 Cal. App. 4th 1">99 Cal. App. 4th 1</a></a> , 10, <a href="https://cite.case.law/cal-app-4th/99/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> (2002) (wrongful eviction is a discrete tort that consists in "the formal termination of a legal relationship"). 35 Sylve v. Riley <a href="https://cite.case.law/cal-app-4th/15/23"><a class="citation" href="https://cite.case.law/cal-app-4th/15/23" title="15 Cal. App. 4th 23">15 Cal. App. 4th 23</a></a> <a href="https://cite.case.law/cal-app-4th/15/23"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/18/608" title="18 Cal. Rptr. 2d 608">18 Cal.Rptr.2d 608</a></a> Menefee v. Ostawari, <a href="https://cite.case.law/cal-app-3d/228/239"><a class="citation" href="https://cite.case.law/cal-app-3d/228/239" title="228 Cal. App. 3d 239">228 Cal. App. 3d 239</a></a> <a href="https://cite.case.law/cal-app-3d/228/239"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./278/805" title="278 Cal. Rptr. 805">278 Cal.Rptr. 805</a></a> Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal. App. 4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> sine qua non see Swain v. California Casualty Insurance Company, <a href="https://cite.case.law/cal-app-4th/99/1"><a class="citation" href="https://cite.case.law/cal-app-4th/99/1" title="99 Cal. App. 4th 1">99 Cal. App. 4th 1</a></a> <a href="https://cite.case.law/cal-app-4th/99/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> The defendants nonetheless argue that the eviction was at least ongoing until the expiration of the 90-day period (to take possession or tender it back to the tenants). 36 But that provision of the San Francisco Rent Ordinance is a "safe harbor" provision that does not render a previous eviction "wrongful" and instead is evidence about whether the landlord was acting in good faith at the time it served the eviction notice. See <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808, 9">id. at 9</a>, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> ("landlords evicted tenants to allow a family move-in but later rented the property to another tenant; court rejected defendants' argument that the eviction "only became illegal after the fact when the [defendants] failed to move into the premises themselves" and held that the insured's conduct, whether lawful or not, inflict[ed] an obvious harm to an important interest - the claimant's interest in a leasehold, a place of shelter, [or] a home....").

___



### Gov't Emps. Ins. Co. v. Nadkarni, <a href="https://cite.case.law/f-supp-3d/391/917"><a class="citation" href="https://cite.case.law/f-supp-3d/391/917" title="391 F. Supp. 3d 917">391 F. Supp. 3d 917</a></a> (N.D. Cal. 2019)

* Do the defendants' references to 'personal injury' and other enumerated torts extend the eviction into the policy period?

* Coverage is based on when the 'offense is committed.'

* In the context of insurance coverage, for torts like wrongful eviction, the offense is considered committed when the tenants departed or when the defendants regained possession of the property.

* Cunningham v. Universal Underwriters was used to support the principle that insurance coverage for enumerated torts such as wrongful eviction is determined by when the offense was committed, which is either when the tenants departed or when the property owners regained possession.

#### Excerpt:

36 See id. <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> The defendants also suggest that the state complaint's references to "personal injury" and other enumerated torts extend the eviction into the policy period. 37 In their reply, they elaborate: 37 The policy covers damages that arise out of property damage, mental or bodily injury, shock, sickness, disease or death, or injury that arises out of false arrest, false imprisonment, wrongful eviction, wrongful detention, malicious prosecution, libel, slander, defamation of character or invasion of the rights of privacy (so long as not thereafter excluded). By grouping all of these potential liability claims together, GEICO purposefully declined to separate them, and provide a separate trigger for each. 38 arise out of 38 They cite no authority to support the argument. Again, coverage is based on when *928 the "offense is committed." Here, at minimum, that is when the tenants departed or when the defendants regained possession of the property. Cunningham , 98 Cal. App. 4th at 1149 , <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> . The same is true for the other enumerated torts. For example, for the tort of malicious prosecution, insurance-coverage cases hold that -- for purposes of insurance policies that measure coverage from the time when the "offense is committed" -- the tort of malicious prosecution occurs when the complaint is filed. Zurich, 188 Cal. App. 3d at 446-47 , <a href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807" title="232 Cal. Rptr. 807">232 Cal.Rptr. 807</a></a> . *928 Cunningham 98 Cal. App. 4th at 1149 <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Zurich, 188 Cal. App. 3d at 446-47 <a href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807" title="232 Cal. Rptr. 807">232 Cal.Rptr. 807</a></a> The defendants also argue that under the 60-day notice, the tenants were not required to vacate the premises until May 12, 2018, which was in the policy period. 39 That argument does not change the outcome for the same reason: the offense was committed before the policy period, meaning, when the tenants left and when the defendants regained possession. These events happened before May 8, 2018, the effective date of the GEICO policy. 39 CONCLUSION

___



In [11]:
from src.utils.output import write_md_to_pdf
from markdown import markdown

In [12]:
markdown_table = markdown(display_df[[
    "citation_reference",
]].to_html(index=True))

In [13]:
html_text = markdown(formatted_string)
html_text += markdown_table

# pdf_file = await write_md_to_pdf(html_text, "pseudo_shep.pdf")

In [14]:
from bs4 import BeautifulSoup

def generate_html_with_toc(html_text: str) -> str:
    """
    Generates HTML content with a table of contents (ToC) based on the headers found in the input HTML text.
    
    Args:
        html_text (str): The HTML content converted from Markdown.
        
    Returns:
        str: The HTML content with a ToC inserted at the beginning.
    """
    soup = BeautifulSoup(html_text, 'html.parser')

    toc_list = soup.new_tag("ul", id='table-of-contents')
    back_to_toc_template = soup.new_tag("a", href="#table-of-contents")
    back_to_toc_template.string = "Back to Table of Contents"
    back_to_toc_template['class'] = 'back-to-toc'

    for header in soup.find_all(['h1', 'h2', 'h3']):
        header_id = header.text.replace(" ", "-").lower()
        header['id'] = header_id  # Assign ID to header for linking
        
        # Create ToC entry
        li = soup.new_tag("li")
        a = soup.new_tag("a", href=f"#{header_id}")
        a.string = header.text
        li.append(a)
        toc_list.append(li)
        
        # Add back-link after the header or section
        back_link = soup.new_tag("a", href="#table-of-contents", **{'class': 'back-to-toc'})
        back_link.string = "Back to Table of Contents"
        header.insert_after(back_link)

    # Check if the body tag exists and insert the ToC, otherwise prepend to the soup object
    if soup.body:
        soup.body.insert(0, toc_list)
    else:
        soup.insert(0, toc_list)

    return str(soup)


In [15]:
html_content_with_toc = generate_html_with_toc(html_text)

In [16]:
Markdown(html_content_with_toc)

<ul id="table-of-contents"><li><a href="#brucia-v.-hartford-accident-&amp;-indemnity,-307-f.-supp.-2d-1079-(n.d.-cal.-2003)">Brucia v. Hartford Accident &amp; Indemnity, 307 F. Supp. 2d 1079 (N.D. Cal. 2003)</a></li><li><a href="#mercury-insurance-v.-ayala,-116-cal.-app.-4th-1198-(cal.-ct.-app.-2004)">Mercury Insurance v. Ayala, 116 Cal. App. 4th 1198 (Cal. Ct. App. 2004)</a></li><li><a href="#f-&amp;-h-construction-v.-itt-hartford-insurance,-118-cal.-app.-4th-364-(cal.-ct.-app.-2004)">F &amp; H Construction v. ITT Hartford Insurance, 118 Cal. App. 4th 364 (Cal. Ct. App. 2004)</a></li><li><a href="#davis-v.-farmers-insurance-group,-134-cal.-app.-4th-100-(cal.-ct.-app.-2005)">Davis v. Farmers Insurance Group, 134 Cal. App. 4th 100 (Cal. Ct. App. 2005)</a></li><li><a href="#demarco-v.-everest-indemnity-insurance,-370-f.-app'x-795-(9th-cir.-2010)">Demarco v. Everest Indemnity Insurance, 370 F. App'x 795 (9th Cir. 2010)</a></li><li><a href="#lakeland-village-homeowners-ass'n-v.-great-american-insurance-group,-727-f.-supp.-2d-887-(e.d.-cal.-2010)">Lakeland Village Homeowners Ass'n v. Great American Insurance Group, 727 F. Supp. 2d 887 (E.D. Cal. 2010)</a></li><li><a href="#btr-hampstead,-llc-v.-source-interlink-distribution,-llc,-194-md.-app.-538-(md.-ct.-spec.-app.-2010)">BTR Hampstead, LLC v. Source Interlink Distribution, LLC, 194 Md. App. 538 (Md. Ct. Spec. App. 2010)</a></li><li><a href="#ginsberg-v.-gamson,-205-cal.-app.-4th-873-(cal.-ct.-app.-2012)">Ginsberg v. Gamson, 205 Cal. App. 4th 873 (Cal. Ct. App. 2012)</a></li><li><a href="#imperium-insurance-v.-unigard-insurance,-16-f.-supp.-3d-1104-(e.d.-cal.-2014)">Imperium Insurance v. Unigard Insurance, 16 F. Supp. 3d 1104 (E.D. Cal. 2014)</a></li><li><a href="#thee-sombrero,-inc.-v.-scottsdale-ins.-co.,-239-cal.-rptr.-3d-416-(cal.-app.-ct.-2018)">Thee Sombrero, Inc. v. Scottsdale Ins. Co., 239 Cal. Rptr. 3d 416 (Cal. App. Ct. 2018)</a></li><li><a href="#mcmillin-homes-constr.,-inc.-v.-nat'l-fire-&amp;-marine-ins.-co.,-247-cal.-rptr.-3d-825-(cal.-app.-ct.-2019)">Mcmillin Homes Constr., Inc. v. Nat'l Fire &amp; Marine Ins. Co., 247 Cal. Rptr. 3d 825 (Cal. App. Ct. 2019)</a></li><li><a href="#gov't-emps.-ins.-co.-v.-nadkarni,-391-f.-supp.-3d-917-(n.d.-cal.-2019)">Gov't Emps. Ins. Co. v. Nadkarni, 391 F. Supp. 3d 917 (N.D. Cal. 2019)</a></li><li><a href="#gov't-emps.-ins.-co.-v.-nadkarni,-391-f.-supp.-3d-917-(n.d.-cal.-2019)">Gov't Emps. Ins. Co. v. Nadkarni, 391 F. Supp. 3d 917 (N.D. Cal. 2019)</a></li></ul><h3 id="brucia-v.-hartford-accident-&amp;-indemnity,-307-f.-supp.-2d-1079-(n.d.-cal.-2003)">Brucia v. Hartford Accident &amp; Indemnity, <a href="https://cite.case.law/f-supp-2d/307/1079"><a class="citation" href="https://cite.case.law/f-supp-2d/307/1079" title="307 F. Supp. 2d 1079">307 F. Supp. 2d 1079</a></a> (N.D. Cal. 2003)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>Does damage to a leasehold interest constitute "property damage" as defined in insurance policy terms, specifically in cases where the tenant has not yet taken possession of the property?</p>
</li>
<li>
<p>A tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession merely gives the tenant a right to abandon the tenancy and sue for damages, but does not grant the tenant the legal right to act as a possessor of the real property until actual transfer. Therefore, damage to a leasehold interest, in cases where possession has not occurred, does not constitute damage to a tangible property right.</p>
</li>
<li>
<p>In the context of insurance policies specifying coverage for "property damage" limited to "tangible property," damage to a leasehold interest, where the tenant has not yet taken possession, falls outside the scope of property damage. This is due to the characterization of a tenant’s interest prior to possession as a contractual rather than a tangible property right.</p>
</li>
<li>
<p>The citation was used to support the argument that damage to a leasehold interest, prior to the tenant taking possession, does not constitute 'property damage' under the defined terms of an insurance policy. It clarifies the legal status of a leasehold interest as initially a contractual right, which does not transform into a tangible property right until actual possession occurs.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>For the foregoing reasons, the Court GRANTS Plaintiffs’ motion for partial summary judgment and DENIES The Hartford’s Motion to Dismiss. IT IS SO ORDERED. . Plaintiffs have also argued that the allegations in the Underlying Complaint cannot be claims for "property damage,” since the property damage section of the policy applies only to "tangible property.” Plaintiffs argue that a leasehold interest is not "tangible property” and therefore damage to a leasehold interest is not property damage as defined in the policy. No California court has directly considered whether a tenant in possession of a property has a tangible property right. In Kazi v. State Farm Fire and Casualty Co., <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/cal-4th/24/871" title="24 Cal. 4th 871">24 Cal.4th 871</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> (2001), the Supreme Court of California held that an easement represents only a nonposses-sory right to use another’s property, and therefore is not tangible property. <a class="citation" href="https://cite.case.law/p3d/15/223#p881" title="15 P.3d 223, 881">Id. at 881</a>, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/p3d/15/223"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> . In Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002), a California appellate court applied the reasoning of Kazi to hold that a nonpossessing tenant, whose claimed loss concerned solely the interference with its right to possession of real property under the parties’ lease, did not allege damage to a tangible property right. As the court in Cunningham explained, "[a] tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer.” <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162, 1155">Id. at 1155</a>-1156, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> . The clear implication of the Cunningham court’s reasoning is that when a tenant legally takes possession of the premises, the tenant, unlike an easement holder or a nonpossessing tenant, has a possessory property right in the premises. As such, a tenant in possession presumably has a tangible property interest in the premises and can assert claims for property damage, as defined in the insurance contract. Regardless, this argument is not dispositive in the instant case, since the Court finds that R &amp; H has asserted claims for personal injury that extend beyond property damage. . Plaintiffs have also argued that the allegations in the Underlying Complaint cannot be claims for "property damage,” since the property damage section of the policy applies only to "tangible property.” Plaintiffs argue that a leasehold interest is not "tangible property” and therefore damage to a leasehold interest is not property damage as defined in the policy. No California court has directly considered whether a tenant in possession of a property has a tangible property right. In Kazi v. State Farm Fire and Casualty Co., <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/cal-4th/24/871" title="24 Cal. 4th 871">24 Cal.4th 871</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> (2001), the Supreme Court of California held that an easement represents only a nonposses-sory right to use another’s property, and therefore is not tangible property. <a class="citation" href="https://cite.case.law/p3d/15/223#p881" title="15 P.3d 223, 881">Id. at 881</a>, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> , <a href="https://cite.case.law/p3d/15/223"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> . In Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002), a California appellate court applied the reasoning of Kazi to hold that a nonpossessing tenant, whose claimed loss concerned solely the interference with its right to possession of real property under the parties’ lease, did not allege damage to a tangible property right. As the court in Cunningham explained, "[a] tenant’s right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer.” <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162, 1155">Id. at 1155</a>-1156, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> . The clear implication of the Cunningham court’s reasoning is that when a tenant legally takes possession of the premises, the tenant, unlike an easement holder or a nonpossessing tenant, has a possessory property right in the premises. As such, a tenant in possession presumably has a tangible property interest in the premises and can assert claims for property damage, as defined in the insurance contract. Regardless, this argument is not dispositive in the instant case, since the Court finds that R &amp; H has asserted claims for personal injury that extend beyond property damage. Kazi v. State Farm Fire and Casualty Co., <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/cal-4th/24/871" title="24 Cal. 4th 871">24 Cal.4th 871</a></a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> <a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">Id.</a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/103/1" title="103 Cal. Rptr. 2d 1">103 Cal.Rptr.2d 1</a></a> <a href="https://cite.case.law/cal-4th/24/871"><a class="citation" href="https://cite.case.law/p3d/15/223" title="15 P.3d 223">15 P.3d 223</a></a> Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Kazi Cunningham <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">Id.</a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Cunningham</p>
<hr/>
<h3 id="mercury-insurance-v.-ayala,-116-cal.-app.-4th-1198-(cal.-ct.-app.-2004)">Mercury Insurance v. Ayala, <a href="https://cite.case.law/cal-app-4th/116/1198"><a class="citation" href="https://cite.case.law/cal-app-4th/116/1198" title="116 Cal. App. 4th 1198">116 Cal. App. 4th 1198</a></a> (Cal. Ct. App. 2004)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>In the context of uninsured motorist coverage, does the per person limit or the per accident limit apply to the appellant's claims? In this case, does the language of the insurance policy clearly define which limit applies</p>
</li>
<li>
<p>If the terms of an insurance policy are unambiguous, the plain meaning of those terms should be given effect.</p>
</li>
<li>
<p>The language of the insurance policy in question was deemed unambiguous by the court, and it was determined that the per person limit applies to the appellant's claims in line with the insurance company's interpretation.</p>
</li>
<li>
<p>The citation was referenced to support the principle that if insurance policy language is unambiguous, the plain meaning of those terms should be given effect. It was used to justify the application of the per person limit in the case at hand, agreeing with the insurance company's interpretation.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<ol>
<li>Coverage: Which limit applies?</li>
</ol>
<p>2</p>
<p>Coverage: Which limit applies? 2 2 ' Part IV of appellants’ policy is entitled “Uninsured Motorists Coverage.” In it, Mercury promises to pay “all sums which the insured . . . shall be legally entitled to recover as damages from the owner, or operator of an uninsured motor vehicle because of bodily injury, sustained by the insured, caused by accident and arising out of the . . . use of such uninsured motor vehicles . . . .” ' The next part of the policy, titled “Part IV—Conditions,” includes the provision that “The limit of liability stated in the declarations as applicable to ‘each person’ is the limit of the company’s liability for all damages arising out of bodily injury sustained by one person in any one accident.... For the purposes of this provision, the ‘bodily injury sustained by any one person’ as used herein, shall be deemed to include all injury and damages for care, loss of consortium and injury to any interpersonal relationship sustained by others as a consequence of such bodily injury.” The second quoted sentence is in boldface. At summary judgment, Mercury contended that the unambiguous language of this clause meant that the per person limit of $15,000, not the per accident limit of $30,000, applies to appellants’ claims. Appellants make two claims, that the per accident limit applies under the terms of the policy and that the per accident limit must apply, because Insurance Code section 11580.2 mandates that it does. As to the policy itself, we agree with Mercury that the language is unambiguous. We thus give effect to its plain meaning (Cunningham v. <em>1202 Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141#p1149"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1149 [ <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ]), which is clearly the one Mercury ascribes to it—the per person limit applies. Numerous cases beginning with United Services Automobile Assn. v. Warner (1976) <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://cite.case.law/cal-app-3d/64/957" title="64 Cal. App. 3d 957">64 Cal.App.3d 957</a></a> [ <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./135/34" title="135 Cal. Rptr. 34">135 Cal.Rptr. 34</a></a> ] have construed similar language in liability policies and have found that the language is unambiguous and means what Mercury, and we, say that it means. (Cunningham </em>1202 Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141120"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> United Services Automobile Assn. v. Warner <a href="https://cite.case.law/cal-app-3d/64/957135"><a class="citation" href="https://cite.case.law/cal-app-3d/64/957" title="64 Cal. App. 3d 957">64 Cal.App.3d 957</a></a> <a href="https://cite.case.law/cal-app-3d/64/957"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./135/34" title="135 Cal. Rptr. 34">135 Cal.Rptr. 34</a></a> Appellants seek a different result, citing Abellon v. Hartford Ins. Co. (1985) <a href="https://cite.case.law/cal-app-3d/167/21"><a class="citation" href="https://cite.case.law/cal-app-3d/167/21" title="167 Cal. App. 3d 21">167 Cal.App.3d 21</a></a> [ <a href="https://cite.case.law/cal-app-3d/167/21"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./212/852" title="212 Cal. Rptr. 852">212 Cal.Rptr. 852</a></a> ]. Abellon, like Warner, is a declaratory relief action filed by an insurer after its insured was in a car accident and was sued not only by the other driver, but by that driver’s spouse on a loss of consortium claim. Neither case involves uninsured motorist coverage, but in both cases, the insurer sought a declaration that the per person, not the per accident, limit of the liability coverage applied. Warner found that the per person limit applied, and Abellon found that the per accident limit applied. However, in Warner, the policy language was like the language before us, 3 and in Abellon there was no similar language defining “bodily injury to any one person” to include the loss of consortium sustained by another. Instead, the insured in Abellon case had “no notice when it purchased the policy that loss of consortium damages fell within the purview of the ‘per person’ limitation.” (Abellon, at p. 31.) Abellon does not help us construe appellants’ policy. Abellon v. Hartford Ins. Co. <a href="https://cite.case.law/cal-app-3d/167/21"><a class="citation" href="https://cite.case.law/cal-app-3d/167/21" title="167 Cal. App. 3d 21">167 Cal.App.3d 21</a></a></p>
<hr/>
<h3 id="f-&amp;-h-construction-v.-itt-hartford-insurance,-118-cal.-app.-4th-364-(cal.-ct.-app.-2004)">F &amp; H Construction v. ITT Hartford Insurance, <a href="https://cite.case.law/cal-app-4th/118/364"><a class="citation" href="https://cite.case.law/cal-app-4th/118/364" title="118 Cal. App. 4th 364">118 Cal. App. 4th 364</a></a> (Cal. Ct. App. 2004)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>Does welding lower grade pile caps to driven piles constitute physical injury to tangible property when the only injury is the structure’s failure to perform as intended?</p>
</li>
<li>
<p>Welding lower grade pile caps to driven piles does not constitute physical injury to tangible property where the only injury shown is the welded structure’s failure to perform as intended.</p>
</li>
<li>
<p>The court considered whether F&amp;H suffered property damage under the definition of loss of use of tangible property that is not physically injured. It concluded that costs for repairing and modifying defective caps and for loss of early completion bonus are unrelated to the rental value and do not qualify as property damage under the commercial liability insurance policy.</p>
</li>
<li>
<p>The court referenced Cunningham v. Universal Underwriters to support the conclusion that cost of repairs and modifications for defective workmanship does not qualify as property damage covered by commercial liability insurance policy, affirming that contractors and developers cannot obtain liability insurance for inferior or defective workmanship - a risk that is not covered by commercial liability insurance. It was used to strengthen the court's reasoning for granting summary judgment in favor of Hartford, indicating that F&amp;H failed to establish it suffered property damage within the meaning of the CGLI policy issued by Hartford.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p><em>377 Collin v. American Empire Ins. Co., supra, Traveler’s We therefore hold that welding the lower grade pile caps to the driven piles does not constitute physical injury to tangible property where the only injury shown is the welded structure’s failure to perform as intended. E. Loss of Use of Tangible Property Loss of Use of Tangible Property We next consider whether F&amp;H suffered “property damage” under the second definition of that term, i.e. “[ljoss of use of tangible property that is not physically injured.” The court in Collin v. American Empire Ins. Co., supra, 21 Cal.App.4th at page 818, explained the meaning of the phrase “loss of use,” quoting from 23 California Jurisprudence Third: “ ‘The measure of damages for the loss of use of personal property may be determined with reference to the rental value of similar property which the plaintiff can hire for use during the period when he is deprived of the use of his own property.’ (23 Cal.Jur.3d, Damages, § 69, pp. 129-130 . . . .)” (Italics omitted.) Collin v. American Empire Ins. Co., supra, F&amp;H does not seek damages for the rental value (or its equivalent) for the loss of the use of the Los Vaqueros facility during the time period modifications were made to the pile caps, or even for a period of time caused by delay. The likely reason for this is the fact the project was completed on time. As noted, the only costs claimed by F&amp;H are the costs for repairing and modifying the defective caps and for loss of the early completion bonus. Those costs are unrelated to rental value. F&amp;H has therefore failed to establish its damages are covered by the policy as “property damage.” A contrary conclusion would allow contractors and developers to obtain liability insurance for inferior or defective workmanship, a risk not covered by commercial liability insurance. (Maryland Casualty Co. v. Reeder, supra, 221 Cal.App.3d at pp. 967-968; see also Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1156 [ <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ].) Accordingly, we conclude the trial court properly granted summary judgment in favor of Hartford on the grounds F&amp;H failed to establish it suffered property damage within the meaning of the CGLI policy issued by Hartford. (Maryland Casualty Co. v. Reeder, supra, Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> </em>378 DISPOSITION *378 The summary judgment in favor of ITT Hartford Insurance Company of the Midwest is affirmed. As the prevailing party, Hartford is awarded its costs on appeal. (Cal. Rules of Court, rule 27(a)(1).) Morrison, J., and Robie, J., concurred. Hartford Insurance Company of the Midwest was erroneously sued as ITT Hartford Insurance Group, Inc. Hartford Insurance Company of the Midwest was erroneously sued as ITT Hartford Insurance Group, Inc.</p>
<hr/>
<h3 id="davis-v.-farmers-insurance-group,-134-cal.-app.-4th-100-(cal.-ct.-app.-2005)">Davis v. Farmers Insurance Group, <a href="https://cite.case.law/cal-app-4th/134/100"><a class="citation" href="https://cite.case.law/cal-app-4th/134/100" title="134 Cal. App. 4th 100">134 Cal. App. 4th 100</a></a> (Cal. Ct. App. 2005)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>When does an insurer have a duty to defend actions against its policy holder?</p>
</li>
<li>
<p>An insurer must defend any action that seeks damages potentially within the coverage of the policy; however, no duty to defend exists when the third party complaint can by no conceivable theory raise a single issue that could bring it within the policy coverage.</p>
</li>
<li>
<p>This citation establishes that an insurer is obligated to defend any lawsuit that seeks damages which could potentially fall within the coverage scope of the policy. However, if the lawsuit could not conceivable raise any issue that would fall under the policy's coverage, then the insurer has no defense obligation.</p>
</li>
<li>
<p>This citation was used to illustrate when an insurer has a duty to defend a lawsuit against its policy holder. Particularly, it highlights the principle that an insurer must defend actions that potentially seek damages within the policy's coverage, while also noting that no duty exists when the complaint cannot raise any issues that would fit under this coverage.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p><a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://cite.case.law/cal-4th/11/1" title="11 Cal. 4th 1">11 Cal.4th 1</a></a> <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> <a href="https://cite.case.law/p2d/900/619"><a class="citation" href="https://cite.case.law/p2d/900/619" title="900 P.2d 619">900 P.2d 619</a></a> (Waller).) (E.M.M.I., Inc. Zurich American Ins. Co. <a href="https://cite.case.law/cal-4th/32/465"><a class="citation" href="https://cite.case.law/cal-4th/32/465" title="32 Cal. 4th 465">32 Cal.4th 465</a></a> <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/9/701"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/9/701" title="9 Cal. Rptr. 3d 701">9 Cal.Rptr.3d 701</a></a> <a href="https://cite.case.law/p3d/84/385"><a class="citation" href="https://cite.case.law/p3d/84/385" title="84 P.3d 385">84 P.3d 385</a></a> (Waller, supra, (<a class="citation" href="https://cite.case.law/p3d/84/385" title="84 P.3d 385">Id.</a> (Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> An insurer must defend any action that seeks damages potentially within the coverage of the policy; however, no duty to defend exists when the third party complaint “ ‘can by no conceivable theory raise a single issue [that] could bring it within the policy coverage.’ ” (Montrose Chemical Corp. v. Superior Court (1993) <a href="https://cite.case.law/cal-4th/6/287#p299"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> , 299-300 [ <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> ], quoting Gray v. Zurich Insurance Co. (1966) <a href="https://cite.case.law/cal-2d/65/263"><a class="citation" href="https://cite.case.law/cal-2d/65/263" title="65 Cal. 2d 263">65 Cal.2d 263</a></a> , 276, fn. 15 [ <a href="https://cite.case.law/cal-2d/65/263"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./54/104" title="54 Cal. Rptr. 104">54 Cal.Rptr. 104</a></a> , <a href="https://cite.case.law/cal-2d/65/263"><a class="citation" href="https://cite.case.law/p2d/419/168" title="419 P.2d 168">419 P.2d 168</a></a> ] (Gray), italics omitted.) Whether a third party action asserts a potentially covered claim under the policy triggering the duty to defend requires us to interpret the language of the insuring agreement and presents a question of law. (Alex Robertson Co. v. Imperial Casualty &amp; Indemnity Co. (1992) <a href="https://cite.case.law/cal-app-4th/8/338#p342"><a class="citation" href="https://cite.case.law/cal-app-4th/8/338" title="8 Cal. App. 4th 338">8 Cal.App.4th 338</a></a> , 342-343 [ <a href="https://cite.case.law/cal-app-4th/8/338"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/10/165" title="10 Cal. Rptr. 2d 165">10 Cal.Rptr.2d 165</a></a> ]; Union Oil Co. v. International Ins. Co. (1995) <a href="https://cite.case.law/cal-app-4th/37/930"><a class="citation" href="https://cite.case.law/cal-app-4th/37/930" title="37 Cal. App. 4th 930">37 Cal.App.4th 930</a></a> , 936 [ <a href="https://cite.case.law/cal-app-4th/37/930"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/4" title="44 Cal. Rptr. 2d 4">44 Cal.Rptr.2d 4</a></a> ].) An insurer’s duty to defend turns on the facts alleged in the underlying complaint or available to the insurer from extrinsic sources at the time the claim is made. (Montrose Chemical Corp. v. Superior Court, supra, <a class="citation" href="https://cite.case.law/cal-4th/6/287#p295" title="6 Cal. 4th 287, 295">6 Cal.4th at p. 295</a>; Gray, supra, <a class="citation" href="https://cite.case.law/cal-2d/65/263#p276" title="65 Cal. 2d 263, 276">65 Cal.2d at p. 276</a>.) The insured has the burden of showing that the claim falls within the scope of coverage and the insurer has the burden of proving that an otherwise covered claim is barred by a policy exclusion. (Travelers Casualty &amp; Surety Co. v. Superior Court (1998) <a href="https://cite.case.law/cal-app-4th/63/1440"><a class="citation" href="https://cite.case.law/cal-app-4th/63/1440" title="63 Cal. App. 4th 1440">63 Cal.App.4th 1440</a></a> , 1453 [ <a href="https://cite.case.law/cal-app-4th/63/1440"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/75/54" title="75 Cal. Rptr. 2d 54">75 Cal.Rptr.2d 54</a></a> ].) “[I]n an action [in which] none of the claims is even potentially covered, the insurer does not have a duty to defend.” (Buss v. Superior Court (1997) <a href="https://cite.case.law/cal-4th/16/35"><a class="citation" href="https://cite.case.law/cal-4th/16/35" title="16 Cal. 4th 35">16 Cal.4th 35</a></a> , 47 [ <a href="https://cite.case.law/cal-4th/16/35"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/65/366" title="65 Cal. Rptr. 2d 366">65 Cal.Rptr.2d 366</a></a> , <a href="https://cite.case.law/cal-4th/16/35"><a class="citation" href="https://cite.case.law/p2d/939/766" title="939 P.2d 766">939 P.2d 766</a></a> ].) (Montrose Chemical Corp. Superior Court <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/p2d/861/1153"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a></p>
<hr/>
<h3 id="demarco-v.-everest-indemnity-insurance,-370-f.-app'x-795-(9th-cir.-2010)">Demarco v. Everest Indemnity Insurance, <a href="https://cite.case.law/f-appx/370/795"><a class="citation" href="https://cite.case.law/f-appx/370/795" title="370 F. App'x 795">370 F. App'x 795</a></a> (9th Cir. 2010)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>Does the insurance policy require the insurer to defend against a suit seeking damages for an act, error, or omission that occurs in the conduct of the Named Insured’s operations?</p>
</li>
<li>
<p>The insurance policy provides that the insurer has a duty to defend against any suit seeking damages for an act, error or omission that "occurs in the conduct of the Named Insured’s operations."</p>
</li>
<li>
<p>The citation does not directly address an instance of misrepresentation within Great Alarm's operations, such as negotiating stock purchases, but supports the broader context of assessing insurance coverage obligations based on the scope of the insured's operations.</p>
</li>
<li>
<p>The citation is referenced to support determining the scope of an insurance company's duty to defend based on the operations of the insured as defined within the policy.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p><a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> <a href="https://cite.case.law/p2d/846/792"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> <a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">Id.</a> <a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">Id.</a> Montrose Chon. Corp. v. Super. Ct., <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> Horace Mann, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> 846 P.2d at 795 Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> see also Waller v. Truck Ins. Exck, Inc., <a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://cite.case.law/cal-4th/11/1" title="11 Cal. 4th 1">11 Cal.4th 1</a></a> <a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> <a href="https://cite.case.law/cal-4th/11/1"><a class="citation" href="https://cite.case.law/p2d/900/619" title="900 P.2d 619">900 P.2d 619</a></a> Here, the relevant policy provides that Everest has a duty to defend against any suit seeking damages for an act, error or omission that “occurs in the conduct of the Named Insured’s operations.” Greater Alarm’s “operations” are defined in the policy as residential and commercial burglar and fire alarm installation and monitoring, medical emergency systems installation and monitoring, “C.C.T.V. installation/service/repair,” and access control. Although some of the alleged misrepresentations were about Great Alarm’s operations, negotiating the stock purchase was not “in the conduct of’ those operations. See Waller, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> , <a class="citation" href="https://cite.case.law/p2d/900/619#p627" title="900 P.2d 619, 627">900 P.2d at 627</a> . Accordingly, we conclude that the arbitration demand does not “reveal a possibility” of coverage under the policy, and thus, Everest had no duty to defend.</p>
<hr/>
<h3 id="lakeland-village-homeowners-ass'n-v.-great-american-insurance-group,-727-f.-supp.-2d-887-(e.d.-cal.-2010)">Lakeland Village Homeowners Ass'n v. Great American Insurance Group, <a href="https://cite.case.law/f-supp-2d/727/887"><a class="citation" href="https://cite.case.law/f-supp-2d/727/887" title="727 F. Supp. 2d 887">727 F. Supp. 2d 887</a></a> (E.D. Cal. 2010)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>In a duty to defend case, what are the required evidentiary standards for the insured and the insurer when moving for summary judgment?</p>
</li>
<li>
<p>An insured moving for summary judgment in a duty to defend case must only show the existence of a potential for coverage, meaning that the underlying claim may fall within policy coverage. Conversely, an insurer moving for summary judgment must establish the absence of any potential for coverage, indicating that the underlying complaint can by no conceivable theory raise a single issue which could bring it within the policy coverage.</p>
</li>
<li>
<p>This sets disparate evidentiary standards for the insured and the insurer in summary judgment motions concerning duty to defend cases, favoring the insured's ability to demonstrate potential coverage under the policy.</p>
</li>
<li>
<p>Cunningham v. Universal Underwriters is cited in the context as establishing the differing burdens of proof required for insureds and insurers when moving for summary judgment in duty to defend cases. This delineation clarifies that an insured must merely show a potential for coverage, whereas an insurer must conclusively prove the absence of any potential for coverage.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>Scottsdale Ins. Co. v. MV Transp., <a href="https://cite.case.law/cal-4th/36/643#p654"><a class="citation" href="https://cite.case.law/cal-4th/36/643" title="36 Cal. 4th 643">36 Cal.4th 643</a></a> , 654, <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/31/147" title="31 Cal. Rptr. 3d 147">31 Cal.Rptr.3d 147</a></a> , <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://cite.case.law/p3d/115/460" title="115 P.3d 460">115 P.3d 460</a></a> (2005) (citation omitted). Further, “[a]ny doubt as to whether the facts give rise to a duty to defend is resolved in the insured’s favor.” Horace Mann Ins. Co. v. Barbara B., <a href="https://cite.case.law/cal-4th/4/1076#p1081"><a class="citation" href="https://cite.case.law/cal-4th/4/1076" title="4 Cal. 4th 1076">4 Cal.4th 1076</a></a> , 1081, <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> , <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> (1993). Scottsdale Ins. Co. v. MV Transp., <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://cite.case.law/cal-4th/36/643" title="36 Cal. 4th 643">36 Cal.4th 643</a></a> <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/31/147" title="31 Cal. Rptr. 3d 147">31 Cal.Rptr.3d 147</a></a> <a href="https://cite.case.law/cal-4th/36/643"><a class="citation" href="https://cite.case.law/p3d/115/460" title="115 P.3d 460">115 P.3d 460</a></a> Horace Mann Ins. Co. v. Barbara B., 4 <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/cal-4th/4/1076" title="4 Cal. 4th 1076">4 Cal.4th 1076</a></a> <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> In a duty to defend case, an insured moving for summary judgment “need only show ‘the existence of a potential for coverage,’ i.e., ‘that the underlying claim may fall within policy coverage.’ ” Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141#p1147"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1147, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002) (citing Montrose Chem. Corp. v. Sup. Ct., <a href="https://cite.case.law/cal-4th/6/287#p300"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> , 300, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> (1993)). Whereas, an insurer moving for summary judgment “ ‘must establish the absence of any ... potential’ for coverage, i.e., that the underlying complaint ‘can by no conceivable theory raise a single issue which could bring <em>891 it within the policy coverage.’ ” <a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">Id.</a> (citing Montrose Chem. Corp., at 300, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> (1993)). “Although the insurer’s burden in moving for summary judgment is greater than the insured’s burden in bringing its own affirmative motion, this disparity merely reflects the substantive law.” <a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">Id.</a> (citation omitted). Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Montrose Chem. Corp. v. Sup. Ct., <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> </em>891 Id.</p>
<hr/>
<h3 id="btr-hampstead,-llc-v.-source-interlink-distribution,-llc,-194-md.-app.-538-(md.-ct.-spec.-app.-2010)">BTR Hampstead, LLC v. Source Interlink Distribution, LLC, <a href="https://cite.case.law/md-app/194/538"><a class="citation" href="https://cite.case.law/md-app/194/538" title="194 Md. App. 538">194 Md. App. 538</a></a> (Md. Ct. Spec. App. 2010)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>What constitutes an actual eviction and what acts by the landlord could be considered as such?</p>
</li>
<li>
<p>An eviction is defined as the act or process of legally dispossessing a person of land or rental property. A wrongful eviction occurs when the person recovering the property had no right to dispossess the other party from the property.</p>
</li>
<li>
<p>In the context of landlord and tenant relations, wrongful eviction occurs when the landlord, without the legal right to do so, dispossesses the tenant of the rental property. This can include acts that physically remove the tenant or prevent the tenant's access to the property, such as changing locks, refusing entry, padlocking entryways, or threats of violence.</p>
</li>
<li>
<p>The case is referenced to affirm the definition of an eviction as the process of legally dispossessing a person of land or rental property, with a focus on wrongful eviction as occurring when there is no legal right to dispossess the other party from the property. The citation plays a role in understanding the legal concept of eviction within the broader discussion on landlord and tenant disputes over actual eviction.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>sub judice. 2 2 any <em>555 Our review of Maryland case law has disclosed no definition of “actual eviction.” Black’s Law Dictionary 35 (6th ed.1990) defines actual eviction as “[a]n actual expulsion of the tenant out of all or some part of the demised premises. A physical ouster or dispossession from the very thing granted or some substantial part thereof.” (Emphasis added). Thompson on Real Property § 41.03(c)(2), entitled “Actual Eviction by the Landlord,” explains: dispossession from the very thing granted or some substantial part thereof.” The landlord will be in breach of the implied covenant of quiet enjoyment if the landlord, or someone for whose acts the landlord is responsible, wrongfully ousts the tenant from the leased premises. The cases refer to this conduct, which can take the form of physical expulsion or a physical exclusion of the tenant from the leased premises, as an actual eviction. Particular acts by the landlord that have been found to be sufficient to constitute actual eviction of a tenant have included changing the locks on the leased premises, refusing to permit the tenant to enter a building in which space has been leased, padlocking the entry to the leased premises, and taking possession of the leased premises with threats of violence against the tenant or members of the tenant’s family if they attempt to reenter. The cases also recognize that the landlord’s eviction efforts may deprive the tenant of only a portion of the leased premises, leaving the tenant in possession of the remainder. If the landlord deprives the tenant of a significant portion of the leased premises, the landlord’s conduct will generally be regarded as constituting a partial actual eviction, and a breach of the implied covenant. (Emphasis added) (footnotes omitted); see also Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal. Rptr.2d 162</a></a> , 168-69 (2002) (“The legal definition of an eviction is the same: an ‘eviction’ is ‘[t]he act or process of legally dispossessing a person of land or rental property.’ (Black’s </em>556 Law Diet. (7th ed.1999) p. 575, col. 2.) A wrongful eviction thus occurs when the person recovering the property had no right to dispossess the other party from the property.” (Alteration in original)); cf. Day v. Watson, <a href="https://cite.case.law/mich/8/535#p536"><a class="citation" href="https://cite.case.law/mich/8/535" title="8 Mich. 535">8 Mich. 535</a></a> , 536 (1860) (“[T]he entry being followed by a continuous possession, which was inconsistent with the possessory title assured to the tenants under the lease, that possession amounts very clearly to an eviction.”). also Cunningham v. Universal Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal. Rptr.2d 162</a></a> *556 wrongful cf. Day v. Watson, <a href="https://cite.case.law/mich/8/535"><a class="citation" href="https://cite.case.law/mich/8/535" title="8 Mich. 535">8 Mich. 535</a></a> The Restatement (Second) of Property: Landlord and Tenant § 6.1 (hereinafter “the Restatement”), provides in pertinent part: Landlord and Tenant</p>
<hr/>
<h3 id="ginsberg-v.-gamson,-205-cal.-app.-4th-873-(cal.-ct.-app.-2012)">Ginsberg v. Gamson, <a href="https://cite.case.law/cal-app-4th/205/873"><a class="citation" href="https://cite.case.law/cal-app-4th/205/873" title="205 Cal. App. 4th 873">205 Cal. App. 4th 873</a></a> (Cal. Ct. App. 2012)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>What is the legal distinction between a claim for breach of the covenant of quiet enjoyment and a constructive eviction claim?</p>
</li>
<li>
<p>A claim for breach of the covenant of quiet enjoyment can be maintained while the tenant remains in possession of the property, whereas a constructive eviction claim must not be brought until the tenant has vacated the property.</p>
</li>
<li>
<p>In Cunningham v. Universal Underwriters, the court held that while both claims deal with interference by the landlord, the inherent difference lies in the status of tenancy - whether the tenant still occupies the property or has left due to eviction.</p>
</li>
<li>
<p>The citation was used to clarify the essential distinction between a breach of the covenant of quiet enjoyment and a constructive eviction claim, highlighting that for the latter, the tenant must have vacated the property before bringing the claim.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./99/86" title="99 Cal. Rptr. 86">99 Cal.Rptr. 86</a> However, in Guntert v. City of Stockton (1976) <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://cite.case.law/cal-app-3d/55/131" title="55 Cal. App. 3d 131">55 Cal.App.3d 131</a></a> [ <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">126 Cal.Rptr. 690</a></a> ] (Guntert), the court distinguished claims in which the landlord has actually or constructively ousted the tenant from those in which the landlord’s interference with the tenant’s enjoyment or use of the property does not lead to ouster. The Guntert court recognized that a tenant has alternative remedies to respond to the landlord’s interference and, instead of vacating, may “stand upon the lease and sue for damages.” (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690, 140">Id. at p. 140</a>.) In Guntert, the plaintiff tenant had not quit the premises, despite the landlord’s service of an invalid and bad faith eviction notice. The court thus concluded: “[The tenant] is not suing for a wrongful, constructive eviction but for the lessor’s unjustified and unauthorized interference with his profitable use of the leased property. The rule requiring ouster or surrender prior to suit for wrongful eviction does not preclude the tenant from his election to stand upon the lease, remain in possession and sue for breach of contract damages.” (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690, 141">Id. at p. 141</a>, italics added.) Guntert v. City of Stockton <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://cite.case.law/cal-app-3d/55/131" title="55 Cal. App. 3d 131">55 Cal.App.3d 131</a></a> <a href="https://cite.case.law/cal-app-3d/55/131"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">126 Cal.Rptr. 690</a></a> (Guntert), Guntert (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">Id.</a> Guntert, for breach of contract damages.” (<a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./126/690" title="126 Cal. Rptr. 690">Id.</a> Subsequent cases have similarly drawn a distinction between a wrongful eviction claim, and one in which the tenant remains in possession of the premises and sues for damages arising out of the landlord’s interference with the tenant’s beneficial use of the leased premises. For example, in Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1153 [ <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ], the court explained: “While a claim for breach of the covenant of quiet enjoyment is similar to a constructive eviction claim, the critical difference is that the latter claim may not be brought until the tenant has vacated the property.” Thus, breach of the implied covenant of quiet enjoyment can be understood as a title encompassing claims for wrongful eviction, and also claims in which the tenant’s use of the premises is disturbed, but the tenant remains in possession. Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> b. Courts have only attached tort liability to wrongful eviction claims Courts have only attached tort liability to wrongful eviction claims It is true, as Ginsberg asserts, that some courts have implicitly or explicitly indicated a tenant may recover punitive damages in connection with a claim for breach of the implied covenant of quiet enjoyment, or identified the claim as a tort. However, all of those cases describe wrongful eviction claims. In other words, they are cases in which the implied covenant of quiet enjoyment <em>899 is breached by a wrongful eviction. For example, Ginsberg relies on Barkett v. Brucato (1953) <a href="https://cite.case.law/cal-app-2d/122/264"><a class="citation" href="https://cite.case.law/cal-app-2d/122/264" title="122 Cal. App. 2d 264">122 Cal.App.2d 264</a></a> [ <a href="https://cite.case.law/cal-app-2d/122/264"><a class="citation" href="https://cite.case.law/p2d/264/978" title="264 P.2d 978">264 P.2d 978</a></a> ] (Barkett). In Barkett, the landlord engaged in a series of egregious acts intended to force the residential tenants to vacate the premises, such as authorizing the removal of the building’s roof during the rainy season, leading to extensive flooding in the tenants’ apartment. (Id. at pp. 271-272.) The tenants eventually vacated the flat. (Id. at p. 272.) They then filed suit, alleging claims for breach of the covenant of quiet enjoyment, negligence, and “conspiracy to willfully and maliciously harass” the tenants. (Id. at p. 266.) A jury returned a general verdict against the landlord, and awarded $5,000 in damages, but did not award any punitive damages. (Ibid.) </em>899 Barkett Brucato</p>
<hr/>
<h3 id="imperium-insurance-v.-unigard-insurance,-16-f.-supp.-3d-1104-(e.d.-cal.-2014)">Imperium Insurance v. Unigard Insurance, <a href="https://cite.case.law/f-supp-3d/16/1104"><a class="citation" href="https://cite.case.law/f-supp-3d/16/1104" title="16 F. Supp. 3d 1104">16 F. Supp. 3d 1104</a></a> (E.D. Cal. 2014)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>What must an insurer demonstrate to attain summary judgment in its favor on the basis of either lack of coverage under the insurance policy or because the contested claim falls within an exclusion under that policy?</p>
</li>
<li>
<p>An insurer seeking summary judgment must establish the absence of any potential for coverage, meaning that the underlying complaint cannot, under any conceivable theory, raise a single issue which might bring it within the policy coverage; or, when the claim is said to be excluded, the insurer must prove that the claim unequivocally falls within an exclusion.</p>
</li>
<li>
<p>In the context of an insurer seeking summary judgment on either the basis of lack of coverage or because the claim falls within an exclusion, the insurer carries the burden of proof. Specifically, the insurer must demonstrate conclusively that there is no possibility of coverage under the policy or that the claim definitely falls within a policy exclusion. Any ambiguity or doubt concerning the duty to defend should be resolved in favor of the insured, indicating a preference towards granting coverage or defense under the policy unless it is clearly unwarranted.</p>
</li>
<li>
<p>The case was cited to support the principle that, when an insurer moves for summary judgment based on the claim of no coverage or an exclusion, the insurer has the burden to unequivocally prove that the claim cannot possibly fit within the scope of the policy coverage or definitely falls within an exclusion. This standard was referenced from the case Montrose Chemical Corporation v. Superior Court, enforcing the stringent criteria an insurer must meet to attain a summary judgment in their favor.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>65 Cal.2d at 267 54 Cal-Rptr. 104 <a href="https://cite.case.law/p2d/419/168"><a class="citation" href="https://cite.case.law/p2d/419/168" title="419 P.2d 168">419 P.2d 168</a></a> Montrose, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/p2d/861/1153"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> Waller, 11 Cal.4th at 19 <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370" title="44 Cal. Rptr. 2d 370">44 Cal.Rptr.2d 370</a></a> <a href="https://cite.case.law/p2d/900/619"><a class="citation" href="https://cite.case.law/p2d/900/619" title="900 P.2d 619">900 P.2d 619</a></a> An insurer moving for summary judgment “ ‘must establish the absence of any ... potential’ for coverage, i.e., that the underlying complaint ‘can by no conceivable theory raise a single issue which could bring it within the policy coverage.’ ” Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141#p1147"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1147, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002) (quoting Montrose Chem. Corp. v. Superior Court, <a href="https://cite.case.law/cal-4th/6/287#p300"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> , 300, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> , <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> (1993)). When an insurer seeks summary judgment “on the ground the claim is excluded,” the insurer has the burden “to prove that the claim falls within an exclusion.” Roberts v. Assurance Co. of America, <a href="https://cite.case.law/cal-app-4th/163/1398#p1406"><a class="citation" href="https://cite.case.law/cal-app-4th/163/1398" title="163 Cal. App. 4th 1398">163 Cal.App.4th 1398</a></a> , 1406, <a href="https://cite.case.law/cal-app-4th/163/1398"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/78/361" title="78 Cal. Rptr. 3d 361">78 Cal.Rptr.3d 361</a></a> (2008) (quotations and citations omitted). “Any doubt as to whether the facts give rise to a duty to defend is resolved in the insured’s favor.” Horace Mann Ins. Co. v. Barbara B., <a href="https://cite.case.law/cal-4th/4/1076#p1081"><a class="citation" href="https://cite.case.law/cal-4th/4/1076" title="4 Cal. 4th 1076">4 Cal.4th 1076</a></a> , 1081, <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/17/210" title="17 Cal. Rptr. 2d 210">17 Cal.Rptr.2d 210</a></a> , <a href="https://cite.case.law/cal-4th/4/1076"><a class="citation" href="https://cite.case.law/p2d/846/792" title="846 P.2d 792">846 P.2d 792</a></a> (1993). Cunningham v. Univ. Underwriters, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Montrose Chem. Corp. v. Superior Court, <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/cal-4th/6/287" title="6 Cal. 4th 287">6 Cal.4th 287</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/24/467" title="24 Cal. Rptr. 2d 467">24 Cal.Rptr.2d 467</a></a> <a href="https://cite.case.law/cal-4th/6/287"><a class="citation" href="https://cite.case.law/p2d/861/1153" title="861 P.2d 1153">861 P.2d 1153</a></a> Roberts v. Assurance Co. of America, <a href="https://cite.case.law/cal-app-4th/163/1398"><a class="citation" href="https://cite.case.law/cal-app-4th/163/1398" title="163 Cal. App. 4th 1398">163 Cal.App.4th 1398</a></a></p>
<hr/>
<h3 id="thee-sombrero,-inc.-v.-scottsdale-ins.-co.,-239-cal.-rptr.-3d-416-(cal.-app.-ct.-2018)">Thee Sombrero, Inc. v. Scottsdale Ins. Co., <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/239/416"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/239/416" title="239 Cal. Rptr. 3d 416">239 Cal. Rptr. 3d 416</a></a> (Cal. App. Ct. 2018)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>Does a tenant have a tangible property interest in leased premises before taking possession?</p>
</li>
<li>
<p>Before a tenant takes possession, the tenant's right to possess the premises is not considered tangible property; it is viewed as a contractual right that does not mature into a property right until possession actually occurs.</p>
</li>
<li>
<p>In the context of insurance and discussions of tangible property interests, the tenant's pre-possession rights are considered contractual and intangible, contrasting with the tangible nature of property interests that comes with possession.</p>
</li>
<li>
<p>The citation was used in making a distinction between the tangible property interest associated with actual possession of leased premises and the intangible nature of a pre-possession contractual right to occupy such premises.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>Hendrickson a loss of the use of their land Hendrickson v. Zurich American Ins. Co. of Illinois supra <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/85/622"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/85/622" title="85 Cal. Rptr. 2d 622">85 Cal.Rptr.2d 622</a></a> Third, we question IPA's statement that "a right to occupy premises is not a tangible property interest." ( Scottsdale Ins. Co. v. International Protective Agency, Inc. , supra , 105 Wash.App. at p. 250 [19 P.3d at p. 1061].) At least under California law, "[a] lease is ... a conveyance of an estate in real property .... [Citation.]" ( Avalon Pacific-Santa Ana, L.P. v. HD Supply Repair &amp; Remodel, LLC (2011) <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://cite.case.law/cal-app-4th/192/1183" title="192 Cal. App. 4th 1183">192 Cal.App.4th 1183</a></a> , 1190, <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/122/417" title="122 Cal. Rptr. 3d 417">122 Cal.Rptr.3d 417</a></a> .) A building is tangible. Dirt is tangible. Hence, a lessee in possession has a tangible property interest in the leased premises. (Compare Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141#p1155"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1155-1156, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> [before tenant takes possession, tenant's right to possess premises is not tangible property; it "is a contractual right that does not mature into a property right until possession actually occurs."] with Riverbank Holding Co., LLC v. N.H. Ins. Co. (E.D. Cal. 2012) 2012 WL 2119046 , at <em>8, 2012 U.S.Dist. LEXIS 78781 , at </em>22 [distinguishing Cunningham ; tenant has tangible property interest in leased premises after tenant "actually took possession of the property."].) Scottsdale Ins. Co. v. International Protective Agency, Inc. supra Avalon Pacific-Santa Ana, L.P. v. HD Supply Repair &amp; Remodel, LLC <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://cite.case.law/cal-app-4th/192/1183" title="192 Cal. App. 4th 1183">192 Cal.App.4th 1183</a></a> <a href="https://cite.case.law/cal-app-4th/192/1183"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/122/417" title="122 Cal. Rptr. 3d 417">122 Cal.Rptr.3d 417</a></a> Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Riverbank Holding Co., LLC v. N.H. Ins. Co. 2012 WL 2119046 2012 U.S.Dist. LEXIS 78781 Cunningham In any event, the issue is not whether, as a technical legal matter, a leasehold is tangible property. Rather, it is whether an insured, reading his or her policy, would understand "tangible property" to include real property that he or she leases. If your leased apartment was rendered uninhabitable by some noxious stench, you would conclude that you had lost the use of tangible property; and if a lawyer said no, actually you had merely lost the use of your intangible lease, you would goggle in disbelief. Most important, though, our view on this point is dictum, because our case is distinguishable. Here, Sombrero is the owner of the property, not a lessee. As such, it plainly has an interest in tangible property.</p>
<hr/>
<h3 id="mcmillin-homes-constr.,-inc.-v.-nat'l-fire-&amp;-marine-ins.-co.,-247-cal.-rptr.-3d-825-(cal.-app.-ct.-2019)">Mcmillin Homes Constr., Inc. v. Nat'l Fire &amp; Marine Ins. Co., <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/247/825"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/247/825" title="247 Cal. Rptr. 3d 825">247 Cal. Rptr. 3d 825</a></a> (Cal. App. Ct. 2019)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>When a policy term has been judicially construed, is it still considered ambiguous?</p>
</li>
<li>
<p>Where a policy term has been judicially construed, it becomes unambiguous.</p>
</li>
<li>
<p>The court applied this rule to the context of the CCC exclusion, indicating it was not ambiguous given its judicial construction.</p>
</li>
<li>
<p>The case was cited in support of the principle that a term that has been judicially construed is not considered ambiguous, further noting that the judicial construction of the term should be incorporated into the policy unless the parties express a contrary intent.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>Having been judicially construed, the CCC exclusion is not ambiguous Where a policy term has been judicially construed, it is not ambiguous. ( County of San Diego v. Ace Property &amp; Casualty Ins. Co. (2005) <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://cite.case.law/cal-4th/37/406" title="37 Cal. 4th 406">37 Cal.4th 406</a></a> , 423, <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/33/583" title="33 Cal. Rptr. 3d 583">33 Cal.Rptr.3d 583</a></a> , <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://cite.case.law/p3d/118/607" title="118 P.3d 607">118 P.3d 607</a></a> ( County of San Diego ).) "[T]he judicial construction of the term should be read into the policy unless the parties express a contrary intent." ( Bartlome v. State Farm (1989) <a href="https://cite.case.law/cal-app-3d/208/1235"><a class="citation" href="https://cite.case.law/cal-app-3d/208/1235" title="208 Cal. App. 3d 1235">208 Cal.App.3d 1235</a></a> , 1239, <a href="https://cite.case.law/cal-app-3d/208/1235"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./256/719" title="256 Cal. Rptr. 719">256 Cal.Rptr. 719</a></a> ( Bartlome ), accord, Cunningham v. Universal Underwriters (2002) <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> , 1150, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> ; County of San Diego v. Ace Property &amp; Casualty Ins. Co. <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/40633/583"><a class="citation" href="https://cite.case.law/cal-4th/37/406" title="37 Cal. 4th 406">37 Cal.4th 406</a></a> <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/33/583" title="33 Cal. Rptr. 3d 583">33 Cal.Rptr.3d 583</a></a> <a href="https://cite.case.law/cal-4th/37/406"><a class="citation" href="https://cite.case.law/p3d/118/607" title="118 P.3d 607">118 P.3d 607</a></a> County of San Diego Bartlome v. State Farm <a href="https://cite.case.law/cal-app-3d/208/1235256"><a class="citation" href="https://cite.case.law/cal-app-3d/208/1235" title="208 Cal. App. 3d 1235">208 Cal.App.3d 1235</a></a> <a href="https://cite.case.law/cal-app-3d/208/1235"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./256/719" title="256 Cal. Rptr. 719">256 Cal.Rptr. 719</a></a> Bartlome Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141120"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal.App.4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> <em>1053 Norris v. Pacific Indem. Co. (1952) <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/cal-2d/39/420" title="39 Cal. 2d 420">39 Cal.2d 420</a></a> , 424, <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/p2d/247/1" title="247 P.2d 1">247 P.2d 1</a></a> ( Norris ) [provisions "should be given a meaning settled by judicial decision"].) This rule is applied " 'with caution, first determining whether the context in which the construed term is analogous ....' " ( Qualcomm, Inc. v. Certain Underwriters at Lloyd's, London (2008) <a href="https://cite.case.law/cal-app-4th/161/184"><a class="citation" href="https://cite.case.law/cal-app-4th/161/184" title="161 Cal. App. 4th 184">161 Cal.App.4th 184</a></a> , 201, <a href="https://cite.case.law/cal-app-4th/161/184"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%203d/73/770" title="73 Cal. Rptr. 3d 770">73 Cal.Rptr.3d 770</a></a> .) As we explain, the CCC exclusion has been judicially construed in a sufficiently analogous context to require exclusive or complete control. ( Davis, supra , 79 Cal.App.3d at p. 872, <a href="https://www.courtlistener.com/c/Cal.%20Rptr./145/158"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./145/158" title="145 Cal. Rptr. 158">145 Cal.Rptr. 158</a></a> .) The additional insured's mere status as general contractor-with overall responsibility for and nominal control of the entire project-does not meet this standard. </em>1053 Norris v. Pacific Indem. Co. <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/cal-2d/39/420" title="39 Cal. 2d 420">39 Cal.2d 420</a></a> <a href="https://cite.case.law/cal-2d/39/420"><a class="citation" href="https://cite.case.law/p2d/247/1" title="247 P.2d 1">247 P.2d 1</a></a> Norris</p>
<hr/>
<h3 id="gov't-emps.-ins.-co.-v.-nadkarni,-391-f.-supp.-3d-917-(n.d.-cal.-2019)">Gov't Emps. Ins. Co. v. Nadkarni, <a href="https://cite.case.law/f-supp-3d/391/917"><a class="citation" href="https://cite.case.law/f-supp-3d/391/917" title="391 F. Supp. 3d 917">391 F. Supp. 3d 917</a></a> (N.D. Cal. 2019)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>In the context of insurance coverage, does the actual departure of a tenant pursuant to an eviction notice constitute wrongful eviction?</p>
</li>
<li>
<p>Physical dispossession (actual departure of a tenant pursuant to an eviction notice) is the sine qua non of eviction.</p>
</li>
<li>
<p>In insurance-coverage cases, the focus is on the actual departure of the tenant, following an eviction notice, as the critical element determining what constitutes eviction.</p>
</li>
<li>
<p>Clarified that physical dispossession is the sine qua non (essential condition) of eviction in the context of insurance coverage disputes regarding wrongful eviction.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>Harbor Ins. Co. v. Cent. Nat'l Ins. Co. <a href="https://cite.case.law/cal-app-3d/165/1029"><a class="citation" href="https://cite.case.law/cal-app-3d/165/1029" title="165 Cal. App. 3d 1029">165 Cal. App. 3d 1029</a></a> <a href="https://cite.case.law/cal-app-3d/165/1029"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./211/902" title="211 Cal. Rptr. 902">211 Cal.Rptr. 902</a></a> Here, as GEICO argues, at least three dates are relevant: March 13, 2018 (when the defendants served the notice of termination of tenancy); April 21, 2018 (when the plaintiffs vacated the property pursuant to the notice); and May 1, 2018 (when the defendants recovered possession of the property). 35 All dates have legal significance, and all occurred before the policy's effective date. For example, cases hold that wrongful eviction occurs when the landlord serves the eviction notice. Sylve v. Riley , <a href="https://cite.case.law/cal-app-4th/15/23#p26"><a class="citation" href="https://cite.case.law/cal-app-4th/15/23" title="15 Cal. App. 4th 23">15 Cal. App. 4th 23</a></a> , 26, <a href="https://cite.case.law/cal-app-4th/15/23"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/18/608" title="18 Cal. Rptr. 2d 608">18 Cal.Rptr.2d 608</a></a> (1993) ; Menefee v. Ostawari, <a href="https://cite.case.law/cal-app-3d/228/239#p246"><a class="citation" href="https://cite.case.law/cal-app-3d/228/239" title="228 Cal. App. 3d 239">228 Cal. App. 3d 239</a></a> , 246, <a href="https://cite.case.law/cal-app-3d/228/239"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./278/805" title="278 Cal. Rptr. 805">278 Cal.Rptr. 805</a></a> (1991) ("Appellant was damaged at the time of service of the 30-day notice in that respondents interfered with his continued right of possession at that time"). In insurance-coverage cases, courts emphasize the actual departure of the tenant pursuant to an eviction notice. Cunningham v. Universal Underwriters , <a href="https://cite.case.law/cal-app-4th/98/1141#p1149"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal. App. 4th 1141</a></a> , 1149, <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> (2002) (physical dispossession is the sine qua non of eviction); see Swain v. California Casualty Insurance Company, <a href="https://cite.case.law/cal-app-4th/99/1#p10"><a class="citation" href="https://cite.case.law/cal-app-4th/99/1" title="99 Cal. App. 4th 1">99 Cal. App. 4th 1</a></a> , 10, <a href="https://cite.case.law/cal-app-4th/99/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> (2002) (wrongful eviction is a discrete tort that consists in "the formal termination of a legal relationship"). 35 Sylve v. Riley <a href="https://cite.case.law/cal-app-4th/15/23"><a class="citation" href="https://cite.case.law/cal-app-4th/15/23" title="15 Cal. App. 4th 23">15 Cal. App. 4th 23</a></a> <a href="https://cite.case.law/cal-app-4th/15/23"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/18/608" title="18 Cal. Rptr. 2d 608">18 Cal.Rptr.2d 608</a></a> Menefee v. Ostawari, <a href="https://cite.case.law/cal-app-3d/228/239"><a class="citation" href="https://cite.case.law/cal-app-3d/228/239" title="228 Cal. App. 3d 239">228 Cal. App. 3d 239</a></a> <a href="https://cite.case.law/cal-app-3d/228/239"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./278/805" title="278 Cal. Rptr. 805">278 Cal.Rptr. 805</a></a> Cunningham v. Universal Underwriters <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://cite.case.law/cal-app-4th/98/1141" title="98 Cal. App. 4th 1141">98 Cal. App. 4th 1141</a></a> <a href="https://cite.case.law/cal-app-4th/98/1141"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> sine qua non see Swain v. California Casualty Insurance Company, <a href="https://cite.case.law/cal-app-4th/99/1"><a class="citation" href="https://cite.case.law/cal-app-4th/99/1" title="99 Cal. App. 4th 1">99 Cal. App. 4th 1</a></a> <a href="https://cite.case.law/cal-app-4th/99/1"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> The defendants nonetheless argue that the eviction was at least ongoing until the expiration of the 90-day period (to take possession or tender it back to the tenants). 36 But that provision of the San Francisco Rent Ordinance is a "safe harbor" provision that does not render a previous eviction "wrongful" and instead is evidence about whether the landlord was acting in good faith at the time it served the eviction notice. See <a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808, 9">id. at 9</a>, <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> ("landlords evicted tenants to allow a family move-in but later rented the property to another tenant; court rejected defendants' argument that the eviction "only became illegal after the fact when the [defendants] failed to move into the premises themselves" and held that the insured's conduct, whether lawful or not, inflict[ed] an obvious harm to an important interest - the claimant's interest in a leasehold, a place of shelter, [or] a home....").</p>
<hr/>
<h3 id="gov't-emps.-ins.-co.-v.-nadkarni,-391-f.-supp.-3d-917-(n.d.-cal.-2019)">Gov't Emps. Ins. Co. v. Nadkarni, <a href="https://cite.case.law/f-supp-3d/391/917"><a class="citation" href="https://cite.case.law/f-supp-3d/391/917" title="391 F. Supp. 3d 917">391 F. Supp. 3d 917</a></a> (N.D. Cal. 2019)</h3><a class="back-to-toc" href="#table-of-contents">Back to Table of Contents</a>
<ul>
<li>
<p>Do the defendants' references to 'personal injury' and other enumerated torts extend the eviction into the policy period?</p>
</li>
<li>
<p>Coverage is based on when the 'offense is committed.'</p>
</li>
<li>
<p>In the context of insurance coverage, for torts like wrongful eviction, the offense is considered committed when the tenants departed or when the defendants regained possession of the property.</p>
</li>
<li>
<p>Cunningham v. Universal Underwriters was used to support the principle that insurance coverage for enumerated torts such as wrongful eviction is determined by when the offense was committed, which is either when the tenants departed or when the property owners regained possession.</p>
</li>
</ul>
<h4>Excerpt:</h4>
<p>36 See id. <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/808" title="120 Cal. Rptr. 2d 808">120 Cal.Rptr.2d 808</a></a> The defendants also suggest that the state complaint's references to "personal injury" and other enumerated torts extend the eviction into the policy period. 37 In their reply, they elaborate: 37 The policy covers damages that arise out of property damage, mental or bodily injury, shock, sickness, disease or death, or injury that arises out of false arrest, false imprisonment, wrongful eviction, wrongful detention, malicious prosecution, libel, slander, defamation of character or invasion of the rights of privacy (so long as not thereafter excluded). By grouping all of these potential liability claims together, GEICO purposefully declined to separate them, and provide a separate trigger for each. 38 arise out of 38 They cite no authority to support the argument. Again, coverage is based on when <em>928 the "offense is committed." Here, at minimum, that is when the tenants departed or when the defendants regained possession of the property. Cunningham , 98 Cal. App. 4th at 1149 , <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> . The same is true for the other enumerated torts. For example, for the tort of malicious prosecution, insurance-coverage cases hold that -- for purposes of insurance policies that measure coverage from the time when the "offense is committed" -- the tort of malicious prosecution occurs when the complaint is filed. Zurich, 188 Cal. App. 3d at 446-47 , <a href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807" title="232 Cal. Rptr. 807">232 Cal.Rptr. 807</a></a> . </em>928 Cunningham 98 Cal. App. 4th at 1149 <a href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162" title="120 Cal. Rptr. 2d 162">120 Cal.Rptr.2d 162</a></a> Zurich, 188 Cal. App. 3d at 446-47 <a href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807"><a class="citation" href="https://www.courtlistener.com/c/Cal.%20Rptr./232/807" title="232 Cal. Rptr. 807">232 Cal.Rptr. 807</a></a> The defendants also argue that under the 60-day notice, the tenants were not required to vacate the premises until May 12, 2018, which was in the policy period. 39 That argument does not change the outcome for the same reason: the offense was committed before the policy period, meaning, when the tenants left and when the defendants regained possession. These events happened before May 8, 2018, the effective date of the GEICO policy. 39 CONCLUSION</p>
<hr/><table border="1" class="dataframe">
<thead>
<tr style="text-align: right;">
<th></th>
<th>citation_reference</th>
</tr>
</thead>
<tbody>
<tr>
<th>0</th>
<td>The citation was used to support the argument that damage to a leasehold interest, prior to the tenant taking possession, does not constitute 'property damage' under the defined terms of an insurance policy. It clarifies the legal status of a leasehold interest as initially a contractual right, which does not transform into a tangible property right until actual possession occurs.</td>
</tr>
<tr>
<th>2</th>
<td>The citation was referenced to support the principle that if insurance policy language is unambiguous, the plain meaning of those terms should be given effect. It was used to justify the application of the per person limit in the case at hand, agreeing with the insurance company's interpretation.</td>
</tr>
<tr>
<th>3</th>
<td>The court referenced Cunningham v. Universal Underwriters to support the conclusion that cost of repairs and modifications for defective workmanship does not qualify as property damage covered by commercial liability insurance policy, affirming that contractors and developers cannot obtain liability insurance for inferior or defective workmanship - a risk that is not covered by commercial liability insurance. It was used to strengthen the court's reasoning for granting summary judgment in favor of Hartford, indicating that F&amp;H failed to establish it suffered property damage within the meaning of the CGLI policy issued by Hartford.</td>
</tr>
<tr>
<th>4</th>
<td>This citation was used to illustrate when an insurer has a duty to defend a lawsuit against its policy holder. Particularly, it highlights the principle that an insurer must defend actions that potentially seek damages within the policy's coverage, while also noting that no duty exists when the complaint cannot raise any issues that would fit under this coverage.</td>
</tr>
<tr>
<th>6</th>
<td>The citation is referenced to support determining the scope of an insurance company's duty to defend based on the operations of the insured as defined within the policy.</td>
</tr>
<tr>
<th>7</th>
<td>Cunningham v. Universal Underwriters is cited in the context as establishing the differing burdens of proof required for insureds and insurers when moving for summary judgment in duty to defend cases. This delineation clarifies that an insured must merely show a potential for coverage, whereas an insurer must conclusively prove the absence of any potential for coverage.</td>
</tr>
<tr>
<th>8</th>
<td>The case is referenced to affirm the definition of an eviction as the process of legally dispossessing a person of land or rental property, with a focus on wrongful eviction as occurring when there is no legal right to dispossess the other party from the property. The citation plays a role in understanding the legal concept of eviction within the broader discussion on landlord and tenant disputes over actual eviction.</td>
</tr>
<tr>
<th>9</th>
<td>The citation was used to clarify the essential distinction between a breach of the covenant of quiet enjoyment and a constructive eviction claim, highlighting that for the latter, the tenant must have vacated the property before bringing the claim.</td>
</tr>
<tr>
<th>10</th>
<td>The case was cited to support the principle that, when an insurer moves for summary judgment based on the claim of no coverage or an exclusion, the insurer has the burden to unequivocally prove that the claim cannot possibly fit within the scope of the policy coverage or definitely falls within an exclusion. This standard was referenced from the case Montrose Chemical Corporation v. Superior Court, enforcing the stringent criteria an insurer must meet to attain a summary judgment in their favor.</td>
</tr>
<tr>
<th>11</th>
<td>The citation was used in making a distinction between the tangible property interest associated with actual possession of leased premises and the intangible nature of a pre-possession contractual right to occupy such premises.</td>
</tr>
<tr>
<th>12</th>
<td>The case was cited in support of the principle that a term that has been judicially construed is not considered ambiguous, further noting that the judicial construction of the term should be incorporated into the policy unless the parties express a contrary intent.</td>
</tr>
<tr>
<th>13</th>
<td>Clarified that physical dispossession is the sine qua non (essential condition) of eviction in the context of insurance coverage disputes regarding wrongful eviction.</td>
</tr>
<tr>
<th>14</th>
<td>Cunningham v. Universal Underwriters was used to support the principle that insurance coverage for enumerated torts such as wrongful eviction is determined by when the offense was committed, which is either when the tenants departed or when the property owners regained possession.</td>
</tr>
</tbody>
</table>

In [23]:
pdf_file_ = await write_md_to_pdf(html_content_with_toc, "pseudo_shep_")

Report written to ./outputs/pseudo_shep_.pdf
